# Big Data FC

The goal of the **Big Data FC** project is to **predict** how many **points** a **football team** belonging to the main European football leagues will end the season with, according to the **characteristics of its players**.

To reach the goal, data relative to the **football players** will first be loaded, in order to then compose the **football teams**.
After that, a second dataset will be used to gather seasonal **rankings**, for every football team.

The project as a whole is composed of:

* This **notebook**, containing all steps of:
  * Data loading.
  * Data cleaning and pre-processing
  * Data visualization.
  * Data analysis.
  * Learning and evaluation.
* A custom [**scraper**](https://github.com/Big-Data-FC/scraper), to gather further players data.
* A set of **REST APIs** to query the loaded data and the prediction model.
* The collection of [scraped datasets](https://github.com/Big-Data-FC/datasets).

During the project, multiple approaches and techniques were explored, all described in this notebook.

_By [Daniele Solombrino](https://github.com/dansolombrino) and [Davide Quaranta](https://github.com/fortym2)._

Before getting started, a series of components that are used across the entire project are imported.

In [1]:
#  PySpark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

import gc

import builtins
import operator
import json

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib widget

import pandas as pd
import numpy as np

from functools import reduce

# import itertools


# import copy

# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.feature import StringIndexer
# from pyspark.ml.feature import StandardScaler
# from pyspark.ml.feature import PCA


# from tqdm import tqdm

# from pyspark.ml.clustering import KMeans
# from pyspark.ml.feature import Normalizer

# RANDOM_SEED = None

# from pyspark.ml.evaluation import ClusteringEvaluator

# from pyspark.ml.linalg import Vectors


# from pyspark.ml.regression import LinearRegression

# from pyspark_dist_explore import hist

# from pyspark.ml.regression import RandomForestRegressor

# from pyspark.ml.regression import DecisionTreeRegressor

# from pyspark.ml import Pipeline

# from pyspark.ml.tuning import ParamGridBuilder

# from pyspark.ml.tuning import CrossValidator
# from pyspark.ml.evaluation import RegressionEvaluator

# from pyspark.ml.regression import GBTRegressor

# from pyspark.ml.classification import MultilayerPerceptronClassifier

# from pyspark.ml.classification import LogisticRegression

# from pyspark.ml.classification import LinearSVC, OneVsRest
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# from pyspark.ml.classification import DecisionTreeClassifier
# from pyspark.ml.classification import RandomForestClassifier

# from operator import itemgetter

# from pyspark.ml.feature import MinMaxScaler

# from pyspark.ml.linalg import Vectors, VectorUDT

# from pyspark.ml.stat import Summarizer

# from sklearn.decomposition import KernelPCA

# from pyspark.ml.feature import UnivariateFeatureSelector
# from pyspark.ml.linalg import DenseVector
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt 
# import seaborn as sns
# import re

# import sklearn
# from sklearn.model_selection import train_test_split
# # from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.preprocessing import scale
# from sklearn.feature_selection import RFE
# # from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import make_pipeline

# from sklearn.model_selection import cross_val_score

# from sklearn.linear_model import Ridge
# from sklearn.linear_model import Lasso
# # from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# # from sklearn.ensemble import RandomForestRegressor

# from sklearn.preprocessing import FunctionTransformer


# import warnings # supress warnings
# warnings.filterwarnings('ignore')

# Spark

The following cell initializes the Spark context, with a given configuration.

In [2]:
conf = (
    SparkConf()
    .set("spark.ui.port", "4050")
    .set("spark.executor.memory", "4G")
    .set("spark.driver.memory", "20G")
    .set("spark.driver.maxResultSize", "10G")
)
# .set("spark.master", "spark://192.168.1.189:4050")


# create the context
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")


spark = SparkSession.builder.getOrCreate()

22/06/21 08:33:51 WARN Utils: Your hostname, RTX-2070-Rig resolves to a loopback address: 127.0.1.1; using 192.168.1.189 instead (on interface wlp7s0)
22/06/21 08:33:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/21 08:33:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Loading football players

First step is to load football players data, which comes from to two different sources:

* For seasons between 2015 and 2020 (called "modern"): [FIFA 15-21 complete dataset](https://www.kaggle.com/datasets/stefanoleone992/fifa-21-complete-player-dataset)

* For season between 2007 and 2014 (called "legacy"): scraped data from [sofifa.com](https://sofifa.com), a website specialized in storing data taken from EA Sports FIFA games.

As introduced before, scraped datasets are committed in a [GitHub repository](https://github.com/Big-Data-FC/datasets).

From now on, the terms "modern" and "legacy" will be used to refer to the two kinds of datasets.

Initially, modern and legacy data will be splitted in two different dataframes, since there are some differences in the structure of their data, among these different years.

In [3]:
modern_df = spark.read.csv(
    "data/players_*.csv", sep=",", inferSchema=True, header=True, multiLine=True
)

legacy_df = spark.read.csv(
    "data/scraped_players_*.csv", sep=",", inferSchema=True, header=True, 
    multiLine=True
)

# Pre-processing football players

In order to focus the project on the major European leagues, it is useful to define a list of leagues to filter, and also to define a list of season to easily discriminate between modern an legacy data.

In [4]:
# These are the European Leagues supported by Big-Data-FC
leagues = [
    "Italian Serie A",
    # "Spain Primera Division",
    # "German 1. Bundesliga",
    # "French Ligue 1",
    # "English Premier League",
    # "Holland Eredivisie",
]

# These are the seasons supported by Big-Data-FC
# seasons_modern = ["20", "19", "18", "17", "16", "15", "14"] 
seasons_modern = ["20"] 
# seasons_legacy = ["13", "12", "11", "10", "09", "08", "07"]
seasons_legacy = ["13"]
seasons = seasons_legacy + seasons_modern

The next definition is about **macro roles**, which is a custom-defined abstration to aggregate affine football roles.

For example, all the midfield roles such as "central midfielder", "advanced midfielder", "left|right wing" can be **grouped together** in the same macro role "midfielder".

Macro roles will be used later on, in a subsequent learning phase. For this reason, much more about them will be touched in future points.

The following cell defines the actual aggregation from FIFA roles abbreviations into macro roles.

In [5]:
macro_roles = ["0.0", "1.0", "2.0", "3.0"]

roles_to_macro_roles_dict = {
    "GK": "0",
    "LB": "1",
    "RB": "1",
    "RWB": "1",
    "LWB": "1",
    "CB": "1",
    "CDM": "2",
    "CM": "2",
    "RM": "2",
    "LM": "2",
    "CAM": "2",
    "RW": "3",
    "LW": "3",
    "ST": "3",
    "LF": "3",
    "RF": "3",
    "CF": "3",
}

NUM_MACRO_ROLES = 4

roles_to_macro_role_UDF = udf(
    lambda roles: float(
        roles_to_macro_roles_dict[roles.split(",")[0]]
    ), 
    StringType()
)

Since the columns associeted to players also contain graphical or data that is generally not informative for the project's purpose (such as shirt number, celebration moves, etc), a list of meaningful columns has been defined, on which the actual working dataframes will be based on.

In [6]:
columns = [
    "short_name",
    "club_name",
    "league_name",
    "season",
    "player_positions",
    "macro_role",
    "overall",
    "value",
    "pace",
    "shooting",
    "passing",
    "dribbling",
    "defending",
    "physic",
    "attacking_crossing",
    "attacking_finishing",
    "attacking_heading_accuracy",
    "attacking_short_passing",
    "skill_dribbling",
    "skill_fk_accuracy",
    "skill_long_passing",
    "skill_ball_control",
    "movement_acceleration",
    "movement_sprint_speed",
    "movement_reactions",
    "power_shot_power",
    "power_stamina",
    "power_strength",
    "power_long_shots",
    "mentality_aggression",
    "mentality_penalties",
    "defending_standing_tackle"
]

Datasets do **not** explicitly include the **year** (season) to which the record refers to.

Rather, this information is implicitly stored in a URL (also for the non-scraped ones, which still originate from the same source), which has its own field.
For this reason, a function to extract such information from aforementioned field is needed.

As an example, a if the URL is `/player/41236/zlatan-ibrahimovic/130034/`, the corresponding season is `13` (from `/13xxxx/`).

In [7]:
def get_season(url):
    url_split = url.split("/")

    # FIFA years must be scaled by a negative factor of one (i.e. 2021 has to be 2020, etc.)
    # This is needed to ensure compatibility with the seasonal score dataset
    return str(
        (int(url_split[-2 if url_split[-1] == "" else -1][0:2]) - 1)
    ).zfill(2)

get_season_UDF = udf(lambda url: get_season(url), StringType())

The format of the **monetary value** of players is different among the modern and legacy dataset.

Specifically, the legacy one abbreviates the values into the form `€10M` to represent `€10000000`.

The following function is used to convert it into the extended one.

In [8]:
@udf
def expand_value_UDF(value):
    value = value.replace("€", "")
    if value[-1] not in ("K", "M"):
        # no abbreviation at the end
        return value

    # extract the number and the unit
    num = value[:-1]
    unit = value[-1]

    # decide based on the unit
    if unit == "M":
        return float(num) * 1000000
    if unit == "K":
        return float(num) * 1000

    return "ERROR"

Now to the actual pre-process.

Some actions are needed by legacy and modern both, whilst other are exclusive to either one.

In [9]:
# Extracting season from the player URL, as per previous cell
pre_processed_modern_df = modern_df.withColumn(
    "season", get_season_UDF(col("player_url"))
)
pre_processed_legacy_df = legacy_df.withColumn(
    "season", get_season_UDF(col("player_url"))
)

# Taking only the players playing for teams in supported Leagues, 
# in the supported seasons
pre_processed_modern_df = pre_processed_modern_df.where(
    (pre_processed_modern_df.league_name.isin(leagues))
    &
    (pre_processed_modern_df.season.isin(seasons_modern))
)
pre_processed_legacy_df = pre_processed_legacy_df.where(
    (pre_processed_legacy_df.league_name.isin(leagues))
    &
    (pre_processed_legacy_df.season.isin(seasons_legacy))
)

# Dropping duplicate players
pre_processed_modern_df = pre_processed_modern_df.dropDuplicates(["player_url"])
pre_processed_legacy_df = pre_processed_legacy_df.dropDuplicates(["player_url"])

# Selected columns have been checked for absence of null/missing data.
# Nevertheless, to ensure compatibility and reusability with other datasets, 
# a null-filling sweep is done
pre_processed_modern_df = pre_processed_modern_df.na.fill(0)
pre_processed_legacy_df = pre_processed_legacy_df.na.fill(0)

# Getting the macro role of the player, according to its field position
pre_processed_modern_df = pre_processed_modern_df.withColumn(
    "macro_role", roles_to_macro_role_UDF(col("player_positions"))
)
pre_processed_legacy_df = pre_processed_legacy_df.withColumn(
    "macro_role", roles_to_macro_role_UDF(col("player_positions"))
)

# Renaming the "value_eur" field to "value" to have compatiblity with legacy
pre_processed_modern_df = pre_processed_modern_df.withColumnRenamed(
    "value_eur", "value"
)

# Convert the monetary value to have compatibility with modern
pre_processed_legacy_df = pre_processed_legacy_df.withColumn(
    "value", expand_value_UDF(col("value"))
)

# Renaming some legacy columns, so as they have the same name as in the modern
pre_processed_legacy_df = pre_processed_legacy_df.withColumnRenamed(
    "pas", "passing"
)
pre_processed_legacy_df = pre_processed_legacy_df.withColumnRenamed(
    "dri", "dribbling"
)
pre_processed_legacy_df = pre_processed_legacy_df.drop(col("defending"))
pre_processed_legacy_df = pre_processed_legacy_df.withColumnRenamed(
    "def", "defending"
)
pre_processed_legacy_df = pre_processed_legacy_df.withColumnRenamed(
    "phy", "physic"
)

pre_processed_legacy_df = pre_processed_legacy_df.withColumnRenamed(
    "sho", "shooting"
)
pre_processed_legacy_df = pre_processed_legacy_df.withColumnRenamed(
    "pac", "pace"
)
pre_processed_legacy_df = pre_processed_legacy_df.withColumnRenamed(
    "bov", "overall"
)

# Keeping only the needed columns.
pre_processed_modern_df = pre_processed_modern_df.select(columns)
pre_processed_legacy_df = pre_processed_legacy_df.select(columns)

Checking whether some monetary values have not been successfully converted.

In [10]:
if pre_processed_legacy_df.select("value").where(col("value") == "ERROR").count() > 0:
    print("WARN: some abbreviated monetary values were not correctly expanded.")

After pre-processing, both dataframes have the same set of columns, so they can be concatenated together.

In [11]:
pre_processed_df = pre_processed_modern_df.unionByName(
    pre_processed_legacy_df
)

## Cleaning up memory

In [12]:
del legacy_df
del modern_df
del pre_processed_legacy_df
del pre_processed_modern_df
gc.collect()

593

This is the end result of this section.

For graphical reasons, only a selection of the columns will be shown, just to give an idea of the structure.

In [13]:
pre_processed_df.select(*columns[0:10]).show()

+---------------+----------+---------------+------+----------------+----------+-------+--------+----+--------+
|     short_name| club_name|    league_name|season|player_positions|macro_role|overall|   value|pace|shooting|
+---------------+----------+---------------+------+----------------+----------+-------+--------+----+--------+
|       C. Terzi|    Spezia|Italian Serie A|    20|              CB|       1.0|     68|  180000|  47|      41|
|      G. Pegolo|  Sassuolo|Italian Serie A|    20|              GK|       0.0|     70|  180000|   0|       0|
|      G. Buffon|  Juventus|Italian Serie A|    20|              GK|       0.0|     82| 2200000|   0|       0|
|        P. Gori| Benevento|Italian Serie A|    20|              GK|       0.0|     62|   30000|   0|       0|
|      G. Pandev|     Genoa|Italian Serie A|    20|          ST, RW|       3.0|     72| 1100000|  44|      74|
|    Bruno Alves|     Parma|Italian Serie A|    20|              CB|       1.0|     72|  625000|  35|      67|
|

The complete list of columns is (again for graphical reasons) printed here in an horizontal format, where each item is a tuple of the form `(field_name, type)`.

In [14]:
print(pre_processed_df.dtypes)

[('short_name', 'string'), ('club_name', 'string'), ('league_name', 'string'), ('season', 'string'), ('player_positions', 'string'), ('macro_role', 'string'), ('overall', 'int'), ('value', 'string'), ('pace', 'int'), ('shooting', 'int'), ('passing', 'int'), ('dribbling', 'int'), ('defending', 'int'), ('physic', 'int'), ('attacking_crossing', 'int'), ('attacking_finishing', 'int'), ('attacking_heading_accuracy', 'int'), ('attacking_short_passing', 'int'), ('skill_dribbling', 'int'), ('skill_fk_accuracy', 'int'), ('skill_long_passing', 'int'), ('skill_ball_control', 'int'), ('movement_acceleration', 'int'), ('movement_sprint_speed', 'int'), ('movement_reactions', 'int'), ('power_shot_power', 'int'), ('power_stamina', 'int'), ('power_strength', 'int'), ('power_long_shots', 'int'), ('mentality_aggression', 'int'), ('mentality_penalties', 'int'), ('defending_standing_tackle', 'int')]


# Building football teams

In [15]:
football_teams_df = pre_processed_df

After having pre-processed the football players, it's time to build the football teams.

For the sake of the learning stage of the project, **teams are differentiated across different seasons**; for example, Real Madrid of 2020 is **different** than Real Madrid of 2018.

A football team is then modeled as **the set of the average of the features of all of its football players**.

In [16]:
# Columns that are considered as features
PLAYER_FEATURES = [
    "overall",
    "value",
    "pace",
    "shooting",
    "passing",
    "dribbling",
    "defending",
    "physic",
    "attacking_crossing",
    "attacking_finishing",
    "attacking_heading_accuracy",
    "attacking_short_passing",
    "skill_dribbling",
    "skill_fk_accuracy",
    "skill_long_passing",
    "skill_ball_control",
    "movement_acceleration",
    "movement_sprint_speed",
    "movement_reactions",
    "power_shot_power",
    "power_stamina",
    "power_strength",
    "power_long_shots",
    "mentality_aggression",
    "mentality_penalties",
    "defending_standing_tackle"
]

# Apposing the avg pre-fix to features
PLAYER_FEATURES_AVG = [
    "avg(" + player_feature + ")" for player_feature in PLAYER_FEATURES
]

# Target variable of the learning stage
TARGET_VARIABLE = "points"

Composing the football team, as introduced before.

In [17]:
football_teams_df = football_teams_df.select(
    "season", "club_name", *PLAYER_FEATURES
).groupBy(
    ["season", "club_name"]
).agg(
    { player_feature: "avg" for player_feature in PLAYER_FEATURES }
)

This is the end result of this section.

For graphical reasons, only a selection of the columns will be shown, just to give an idea of the structure.

In [18]:
football_teams_df.select(
    "season", "club_name", *PLAYER_FEATURES_AVG[0:5]
).show()

+------+-------------+-----------------+--------------------+------------------+------------------+------------------+
|season|    club_name|     avg(overall)|          avg(value)|         avg(pace)|     avg(shooting)|      avg(passing)|
+------+-------------+-----------------+--------------------+------------------+------------------+------------------+
|    20|      Udinese|70.15151515151516|  3373636.3636363638|55.484848484848484| 47.54545454545455| 51.60606060606061|
|    20|     Cagliari|           69.625|          3688281.25|           58.4375|          51.90625|          55.46875|
|    20|     Juventus|77.48387096774194| 1.691774193548387E7|  64.6774193548387|55.064516129032256| 62.32258064516129|
|    20|        Genoa|70.45454545454545|   3045151.515151515| 58.03030303030303|51.333333333333336| 56.90909090909091|
|    20|        Lazio|75.51515151515152|1.0902272727272727E7| 65.15151515151516|              55.0| 60.09090909090909|
|    20|     Sassuolo| 70.6969696969697|   42980

The complete list of columns with their type is (in a compact horizontal form):

In [19]:
print(football_teams_df.dtypes)

[('season', 'string'), ('club_name', 'string'), ('avg(attacking_short_passing)', 'double'), ('avg(shooting)', 'double'), ('avg(power_stamina)', 'double'), ('avg(skill_long_passing)', 'double'), ('avg(power_strength)', 'double'), ('avg(defending_standing_tackle)', 'double'), ('avg(skill_fk_accuracy)', 'double'), ('avg(skill_dribbling)', 'double'), ('avg(dribbling)', 'double'), ('avg(pace)', 'double'), ('avg(mentality_aggression)', 'double'), ('avg(movement_reactions)', 'double'), ('avg(movement_sprint_speed)', 'double'), ('avg(passing)', 'double'), ('avg(movement_acceleration)', 'double'), ('avg(attacking_heading_accuracy)', 'double'), ('avg(attacking_finishing)', 'double'), ('avg(defending)', 'double'), ('avg(attacking_crossing)', 'double'), ('avg(power_long_shots)', 'double'), ('avg(mentality_penalties)', 'double'), ('avg(overall)', 'double'), ('avg(power_shot_power)', 'double'), ('avg(value)', 'double'), ('avg(physic)', 'double'), ('avg(skill_ball_control)', 'double')]


# Loading football teams seasonal scores

After having dealt with the football players and composed them into football clubs, it's time to get **"target" data** for the learning stage: the final ranking, for every team of every year.

First step is to load the data from disk, which has been taken from the [European Football Dataset](https://www.kaggle.com/datasets/josephvm/european-club-football-dataset)

In [20]:
seasonal_scores_df = (
    spark.read.csv(
        "data/all_tables_fixed_renamed_leagues.csv",
        sep=",",
        inferSchema=True,
        header=True,
        multiLine=True,
    )
    .withColumnRenamed("Year", "season")
    .withColumnRenamed("Team", "club_name")
    .withColumnRenamed("P", "points")
    .withColumnRenamed("Place", "place")
    .withColumnRenamed("League", "league")
)

# Pre-processing football teams seasonal scores

Out of all the available columns, only the ones in `seasonal_scores_columns` will be taken into consideration.

In [21]:
seasonal_scores_columns = [
    "season", "league", "club_name", "points", "place"
]

The seasonal scores dataset uses **abbreviated version of football team names** (for example: `BAR` for Barcelona, `LEI` for Leicester, etc.), which would cause **incompatibility** with modern and legacy FIFA datasets, which instead uses complete names.

Furthermore, the abbreviations are **not standard**, so there are **conflicts** such as, among others:

* `BAR` both for Barcelona and Bari (different leagues).
* `HUE` both for Huelva and Huesca (same league).

For this reason, a **custom** hand-made **mapping** procedure has been developed in order to resolve such conflicts.

Another source of incompatibility originated from **inconsistencies** within FIFA datasets, from year to year; for example, some teams had slight variations in their names (e.g. Torino and Torino FC).

All said inconsistencies have been **manually solved** and disambiguated at the dataset-level.

Furthermore, the final **mapping** between abbreviations and FIFA names resulted into a **custom-made dataset** (also available in the linked dataset GitHub repo), which is the following form:

|abbr|league|club_name|fifa_club_name|
|---|---|---|---|
|AAC|German Bundesliga|Aachen|Alemannia Aachen|
|ADO|Dutch Eredivisie|Ado Den Haag|ADO Den Haag|
|AJA|Dutch Eredivisie|Ajax|Ajax|
|AJC|French Ligue 1|Ajaccio|AC Ajaccio|
|ALB|Spanish La Liga|Albacete|Albacete BP|
|...|...|...|...|

Aforementioned mapping is then converted to **JSON** with an utility script in the same repository, and loaded here in a dataframe.

In [22]:
f = open("data/clubs_map.json")
club_name_abbr_to_ext = json.load(f)
f.close()

ABBREVIATED_CLUB_NAME_NOT_FOUD = "ABBREVIATED_CLUB_NAME_NOT_FOUD"
GENERAL_EXCEPTION = "GENERAL_EXCEPTION"

def extend_club_name(club_name_abbr):
    try:
        return club_name_abbr_to_ext[club_name_abbr]
    except KeyError as e:
        return ABBREVIATED_CLUB_NAME_NOT_FOUD
    except Exception as e:
        return GENERAL_EXCEPTION

extend_club_name_UDF = udf(
    lambda club_name_abbr: extend_club_name(str(club_name_abbr)),
    StringType(),
)

In the rankings dataset the seasons are expressed as `YYYY`, whilst FIFA uses the `YY` encoding.

For this reason, to guarantee compatibility, season values in the rankings dataset is abbreviated.

In [23]:
abbreviate_season_UDF = udf(
    lambda season: str(season)[-2:],
    StringType(),
)

The following is the actual **data pre-processing**.

In [24]:
pre_processed_seasonal_scores_df = seasonal_scores_df

# Abbreviating season, as per previous cell
pre_processed_seasonal_scores_df = pre_processed_seasonal_scores_df.withColumn(
    "season", abbreviate_season_UDF(col("season"))
)

# Keeping only supported leagues in supported seasons
pre_processed_seasonal_scores_df = pre_processed_seasonal_scores_df.where(
    (pre_processed_seasonal_scores_df.season.isin(seasons))
    & 
    (pre_processed_seasonal_scores_df.league.isin(leagues))
)

# Selecting only the desired columns
pre_processed_seasonal_scores_df = pre_processed_seasonal_scores_df.select(
    seasonal_scores_columns
)

# Although data has been checked for duplicates and missing value, to ensure 
# operabiloty with other datasets, the pre-processing steps are still performed
pre_processed_seasonal_scores_df = pre_processed_seasonal_scores_df.dropDuplicates(
    seasonal_scores_columns
)
pre_processed_seasonal_scores_df = pre_processed_seasonal_scores_df.na.fill(0)

# Extending club names, as per previous cell
pre_processed_seasonal_scores_df = pre_processed_seasonal_scores_df.withColumn(
    "club_name", extend_club_name_UDF(col("club_name"))
)
# Checking whether club name expansions went all good or not
if pre_processed_seasonal_scores_df.filter(
    col("club_name") == ABBREVIATED_CLUB_NAME_NOT_FOUD
).count() > 0:
    print("WARN: some clubs have NOT been found")
    print("Please check your data")

# Casting points to float, as required by learning procedures
pre_processed_seasonal_scores_df = pre_processed_seasonal_scores_df.withColumn(
    "points", pre_processed_seasonal_scores_df.points.cast(DoubleType())
)

This is the end result of this section:

In [25]:
pre_processed_seasonal_scores_df.show()

+------+---------------+-------------+------+-----+
|season|         league|    club_name|points|place|
+------+---------------+-------------+------+-----+
|    20|Italian Serie A|    Sampdoria|  52.0|    9|
|    20|Italian Serie A|Hellas Verona|  45.0|   10|
|    13|Italian Serie A|      Udinese|  44.0|   13|
|    13|Italian Serie A|Hellas Verona|  54.0|   10|
|    13|Italian Serie A|       Torino|  57.0|    7|
|    13|Italian Serie A|    Sampdoria|  45.0|   12|
|    20|Italian Serie A|      Bologna|  41.0|   12|
|    13|Italian Serie A|   Fiorentina|  65.0|    4|
|    20|Italian Serie A|        Genoa|  42.0|   11|
|    20|Italian Serie A|    Benevento|  33.0|   18|
|    13|Italian Serie A|         Roma|  85.0|    2|
|    13|Italian Serie A|        Lazio|  56.0|    9|
|    20|Italian Serie A|      Udinese|  40.0|   14|
|    20|Italian Serie A|        Inter|  91.0|    1|
|    13|Italian Serie A|      Bologna|  29.0|   19|
|    20|Italian Serie A|         Roma|  62.0|    7|
|    13|Ital

# Joining football teams features with their seasonal scores

The two dataframes (players and seasonal scores) need to be merged together to form a single dataframe.
This can easily be done by joining on the key (`season`, `club_name`).

In [26]:
df = football_teams_df.join(
    pre_processed_seasonal_scores_df,
    on = ["season", "club_name"],
)

In order to check whether some clubs were left out by the aforementioned join operation, two differences are computed:

* Seasonal scores dataframe - joined dataframe.
* Football teams dataframe - joined dataframe.

In [27]:
diff = pre_processed_seasonal_scores_df.select("club_name").subtract(df.select("club_name")).distinct()

if diff.count() > 0:
    print("WARN: Some football teams have been left out the join (pre_processed_seasonal_scores_df)")
    diff.show()

diff = football_teams_df.select("club_name").subtract(df.select("club_name")).distinct()
if diff.count() > 0:
    print("WARN: Some football teams have been left out the join (football_teams_df)")
    diff.show()

del diff

This is the end result of this section.

For graphical reasons, only a selection of the columns will be shown, just to give an idea of the structure.

In [28]:
df.select(
    "season", "league", "club_name", "avg(overall)", "avg(pace)", "points", "place"
).show()

+------+---------------+-------------+-----------------+------------------+------+-----+
|season|         league|    club_name|     avg(overall)|         avg(pace)|points|place|
+------+---------------+-------------+-----------------+------------------+------+-----+
|    20|Italian Serie A|      Udinese|70.15151515151516|55.484848484848484|  40.0|   14|
|    20|Italian Serie A|     Cagliari|           69.625|           58.4375|  37.0|   16|
|    20|Italian Serie A|     Juventus|77.48387096774194|  64.6774193548387|  78.0|    4|
|    20|Italian Serie A|        Genoa|70.45454545454545| 58.03030303030303|  42.0|   11|
|    20|Italian Serie A|        Lazio|75.51515151515152| 65.15151515151516|  68.0|    6|
|    20|Italian Serie A|     Sassuolo| 70.6969696969697|60.666666666666664|  62.0|    8|
|    20|Italian Serie A|       Torino|71.06060606060606| 56.54545454545455|  37.0|   17|
|    20|Italian Serie A|       Spezia|65.48387096774194|52.806451612903224|  39.0|   15|
|    20|Italian Serie

The full list of columns with their type is (in a compact horizontal form):

In [29]:
print(df.dtypes)

[('season', 'string'), ('club_name', 'string'), ('avg(attacking_short_passing)', 'double'), ('avg(shooting)', 'double'), ('avg(power_stamina)', 'double'), ('avg(skill_long_passing)', 'double'), ('avg(power_strength)', 'double'), ('avg(defending_standing_tackle)', 'double'), ('avg(skill_fk_accuracy)', 'double'), ('avg(skill_dribbling)', 'double'), ('avg(dribbling)', 'double'), ('avg(pace)', 'double'), ('avg(mentality_aggression)', 'double'), ('avg(movement_reactions)', 'double'), ('avg(movement_sprint_speed)', 'double'), ('avg(passing)', 'double'), ('avg(movement_acceleration)', 'double'), ('avg(attacking_heading_accuracy)', 'double'), ('avg(attacking_finishing)', 'double'), ('avg(defending)', 'double'), ('avg(attacking_crossing)', 'double'), ('avg(power_long_shots)', 'double'), ('avg(mentality_penalties)', 'double'), ('avg(overall)', 'double'), ('avg(power_shot_power)', 'double'), ('avg(value)', 'double'), ('avg(physic)', 'double'), ('avg(skill_ball_control)', 'double'), ('league', 'st

## Cleaning up memory

In [30]:
del seasonal_scores_df
del pre_processed_seasonal_scores_df
del football_teams_df
gc.collect()

388

# Visualizations

A good practice whenever a data-driven task is being taclked is to first visualize the data.

In fact, even just by simply looking at data we may find some useful information which may have a direct impact on the subsequent learning phase.

## Points distribution

Kicking the visualization stage off with data distirbutions for the end-of-season points, across all supported seasons, for all supported leagues.

Data distribution is very important to monitor: exposing a model too many or too few time to a value may be source of bias, which we want to avoid.

In [ ]:
p = sns.displot(df.select("points").toPandas(), kde=True)
p.fig.tight_layout(pad=1.5)
plt.title("Points distribution across all leagues")

Some observations can be done:

1. Data tends to approximatively follow a Gaussian/Normal distribution
2. Said distribution appears to be slighlty skewed towards left

These observations perfectly coincide with domain knowledge.
1. In every league, there are just a few very strong teams (Champions League qualifiers), a limited number of very bad teams (fighting for relegation) and then a multitude of middle-table teams
2. Some leagues do not have much talent in the middle part of the table, resulting in a general "equilibrium" between such team.
As a result, scores of such teams not be high, but rather close to the mean value. 

The following density plot instead focuses on each league, showing again the points distribution.

In [ ]:
p = sns.displot(df.toPandas(), x="points", hue="league", kind="kde", aspect=1.7, palette="tab10")
p.fig.tight_layout(pad=1.5)
plt.title("Points distribution per league")

Some observations can be made:

* The **French** and **Spanish** leagues are the less skewed among all leagues, possibly meaning that they are characterized by a majority of "average" teams.
* The **Italian** and **English** leagues are very similar on low and middle points, but start to behave slightly different on the middle-high and high parts of the table.
* The **Dutch** league is the most left-skewed, possibly hinting at the lower quality of the league, as suggested by the domain knowledge.
* The **Spanish** league has a boost on high points, hinting at a consistent dominancy of one or more teams.

## Overall player quality distribution

Similar visualizations can be done to analyze the overall player quality.

The following plot shows the distribution of the player quality (`avg(overall)`) across all leagues.

In [ ]:
p = sns.displot(df.select("avg(overall)").toPandas(), kde=True)
p.fig.tight_layout(pad=1.5)
plt.title("Player quality across all leagues")

Similarily to what observed for the points, the plot shows that:

* A vast majority of the players are of average quality.
* A minority of the players are below-average.
* A minority of the players are above-average.

The following plot focuses on the single leagues.

In [ ]:
p = sns.displot(df.toPandas(), x="avg(overall)", hue="league", kind="kde", aspect=1.7, palette="tab10")
p.fig.tight_layout(pad=1.5)
plt.title("Player quality per league")

From the plot it is possible to observe that:

* The Dutch league hosts the majority of below-average players.
* The French league follows the Dutch one in below-average players.
* The German league has the highest amount of average players.
* The German, Engligh, Italian and Spanish leagues have a peak of average players.
* The same leagues have a considerable amount of above-average players.
* The Spanish leagues dominates on the above-average players.

# Attempt 1: "naive" player features

The following section represents the first attempt at building a model able to predict the points of a team, considering the features of its players.

This attempt has been renamed as "naive", meaning that it simply consist of an analysis based on **all the features** that are related to a **player's technical abilities**.

In [31]:
# remove columns that do not inherently represent players abilities
ALL_FEATURES = PLAYER_FEATURES_AVG
ALL_FEATURES.remove("avg(overall)")
ALL_FEATURES.remove("avg(value)")


The learning procedures require that the considered features are assembled in a vector.

The following cell performs this operation by adding a new "assembled" column to the main dataframe.

In [32]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=ALL_FEATURES, outputCol="all_vec"
)

df = assembler.transform(df)

It is convenient to define a some custom utility functions to plot multiple figures, that will be called multiple times along the course of the project.

The following cells contain the definition of said functions.

In [ ]:
def plot_feature_target_relation(
    data, x, y, n_cols = 2, figsize = (15, 30), color = "#000000"
):

    n_rows = int(len(x) / n_cols) if len(x) >= n_cols else n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)

    for x_ind, x_value in enumerate(x):
        ax = sns.regplot(
            data=data,
            x=x_value,
            y=y,
            color = color,
            ax=axes[x_ind // n_cols, x_ind % n_cols] if n_rows > 1 else axes,
        )


    fig.tight_layout()

In [ ]:
def plot_feature_distribution(
    data,
    features,
    figsize=(4,4), 
    color="#000000",
    n_cols=2
):

    n_rows = int(len(features) / n_cols) if len(features) >= n_cols else n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)

    for feature_ind, feature in enumerate(features):
        _ = sns.histplot(
            data[feature],
            kde=True,
            color=color,
            facecolor=color,
            ax=axes[feature_ind // n_cols, feature_ind % n_cols] if n_rows > 1 else axes,
        )

    fig.tight_layout(pad=1.5)

In [ ]:
def plot_correlation_matrix(
    data, features, title="Pearson Correlation Matrix", figsize=(16,12)
):

    mask = np.zeros_like(data[features].corr(), dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    with sns.axes_style("white"):  # Temporarily set the background to white
        fig, ax = plt.subplots(figsize=figsize)
        plt.title(title, fontsize=24)

        cmap = sns.diverging_palette(220, 10, as_cmap=True)

        _ = sns.heatmap(
            data[features].corr(),
            linewidths=0.25,
            vmax=0.7,
            square=True,
            ax=ax,
            cmap=cmap,
            linecolor="w",
            annot=True,
            annot_kws={"size": 8},
            mask=mask,
            cbar_kws={"shrink": 0.9},
        )

Before learning, it is good practice to first gather in-depth information on the structure of the data, as well as obtaining useful visualizations.

In the following sections, multiple approaches at data visualization will be given.

## Raw data

In this section a first analysis is performed on _raw_ data, meaning that no normalization or standardization is performed.

In [ ]:
# color to be used in the plots for raw data
COLOR_RAW = "#332FD0"

In [ ]:
# use Pandas for easy plotting with Seaborn
pdf = df.toPandas()

### Feature-target correlation

The **feature-target correlation** shows the correlation between each feature and the target variable (point).

The idea is to immediately visualize whether there are specific features that are particularly correlated with the target variable.

In [ ]:
plot_feature_target_relation(
    pdf, ALL_FEATURES, TARGET_VARIABLE, color=COLOR_RAW
)

Since there are too many datapoints, it is very hard to look at the plots. Let's see how visualizations change when focusing on some leagues or years, thus restricting the data scope.

In [ ]:
plot_feature_target_relation(
    pdf[pdf["season"] == "20"], ALL_FEATURES, TARGET_VARIABLE, color=COLOR_RAW
)

It can be observed that:

1. There is a general phenomenon of linear relation between each feature and the final points.
2. There is a generalized high variance across all features.
   1. E.g. taking the very last feature (`avg(defending_standing_tackle`), players with a value close to `50` result to a wide range of points.
   2. On the contrary, in extreme values (close to `40` or `90`), the resulting variable have a narrower range.

### Feature distribution

The following plot shows the distribution of the player features in the analyzed dataframe.

In [ ]:
plot_feature_distribution(pdf, ALL_FEATURES, color = COLOR_RAW)

From the plot it is possible to observe that:

1. Most features have a symmetrical distribution.
2. Most features are centered around the middle.
3. Some features are more skewed towards left, hinting that they may be more "rare".
4. Some features don't properly follow a Gaussian distribution (`avg(pace)`, `avg(physic)`).

Furthermore, observation `2` is linked to the observation `2.2` of the previous plot:

* A lot of features have an average value.
* Around the average values, there is high variance.

Hence, most likely, the system will be exposed multiple times to this issue, affecting performances.

### Pearson Correlation Matrix

The **Pearson's Correlation Matrix** is a graphical tool to show a numerical value indicating the correlation between each variable.

It is employed in the project, to find whether some features are correlated to each other, which can either be a problem for Linear Regression, but not so much for Tree-based models, since they by design perform a feature selection step.

In [ ]:
plot_correlation_matrix(pdf, ALL_FEATURES)

There are:

* A lot of positive correlations (red).
* Some negative correlations (teal).
* Some feature are tendentially not correlated to others (white).

Using some domain knowledge, it is possible to further comment some correlations, like (among others):

* The more a defender is good, the more aggressive they are.
* The more a defender is good, the more able to perform standing tackles they are.
* The more a player has good physical strenght, the more fast (pace) they are.

## Standardization

Since we have some features with skewed distributions, it is needed to standardize, to see whether it helps to center feature distribution.

In [ ]:
# standardized_df = df

It is needed to define a scaler which takes feature from `all_vec` and places scaled versions in `all_vec_std`.

The configuration is of a standard z-score normalization.

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(
    inputCol="all_vec", 
    outputCol="all_vec_std", 
    withStd=True, 
    withMean=True
)

df = scaler.fit(df).transform(df)

In [ ]:
COLOR_STD = "#9254C8"

ALL_FEATURES_STD = [
    player_feature + "_std" for player_feature in ALL_FEATURES
]

In [ ]:
pdf = df.toPandas()

Since the scaler grouped all features in a single field (feature vector), but plotting requires using different fields (one per features), it is needed to define a function to extract each feature and place it in a separate column.

In [ ]:
def feature_vec_to_cols(pdf, vec, columns):
    tmp = pdf.reindex(
        columns=list(pdf.columns) + columns
    )

    tmp[columns] = tmp[
        vec
    ].transform(
        {
            columns[i]: operator.itemgetter(i) for i, p in enumerate(columns)
        }
    )
    return tmp

In [ ]:
pdf = feature_vec_to_cols(pdf, "all_vec_std", ALL_FEATURES_STD)

It is possible to see the effects of the function by looking at the columns list.

In [ ]:
pdf.columns

### Feature-target correlation

As done in the _raw data_ section, a feature-target correlation analysis is performed.

In [ ]:
plot_feature_target_relation(
    pdf, ALL_FEATURES_STD, TARGET_VARIABLE, color=COLOR_STD
)

Let's again zoom on a single season.

In [ ]:
plot_feature_target_relation(
    pdf[pdf["season"] == "20"], ALL_FEATURES_STD, TARGET_VARIABLE, color=COLOR_STD
)

Standardization does not improve the variance issue.

#### Feature distribution

In [ ]:
plot_feature_distribution(
    pdf, ALL_FEATURES_STD, color = COLOR_STD
)

Standardization did not change the feature distribution.

It is not needed to plot the correlation matrix again, since it is not affected by a change of scale.

## Log transformation

A further attempt consists of apply Logarithmic Transformation, still trying to combat skewness.

In [ ]:
COLOR_LOG = "#E15FED"
ALL_FEATURES_LOG = [
    player_feature + "_log" for player_feature in ALL_FEATURES
]


The following function translates a value to its $log_2$.

In [ ]:
to_log_UDF = udf(
    lambda value: float(np.log2(value)), DoubleType()
)

In [ ]:
for feature, feature_log in zip(ALL_FEATURES, ALL_FEATURES_LOG):
    df = df.withColumn(feature_log, to_log_UDF(col(feature)))

df = df.withColumn("points_log", to_log_UDF(col("points")))

In [ ]:
pdf = df.toPandas()

### Feature-target correlation

In [ ]:
plot_feature_target_relation(
    pdf, ALL_FEATURES_LOG, "points_log", color=COLOR_LOG
)

Also Logarithmic transformation did not help with the variance problem.

Let's again see a zommed version.

In [ ]:
plot_feature_target_relation(
    pdf[pdf["season"] == "20"], ALL_FEATURES_LOG, "points_log", color=COLOR_LOG
)

Logarithmic transformation did not help with variance.

### Feature distribution

In [ ]:
plot_feature_distribution(pdf, ALL_FEATURES_LOG, color = COLOR_LOG)

Logarithmic transformation slightly moved the skewdness of all features towards right, still not solving it; as a consequence, centered or right-skewed features are not slightly worsened.

## Min-max transformation

In [33]:
COLOR_MIN_MAX = "#6EDCD9"
ALL_FEATURES_MIN_MAX = [
    player_feature + "_min_max" for player_feature in ALL_FEATURES
]


In [34]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(
    inputCol="all_vec", 
    outputCol="all_vec_min_max"
)

df = scaler.fit(df).transform(df)

In [ ]:
pdf = df.toPandas()
pdf = feature_vec_to_cols(pdf, vec="all_vec_min_max", columns=ALL_FEATURES_MIN_MAX)

### Feature-target relationship

In [ ]:
plot_feature_target_relation(
    pdf, ALL_FEATURES_MIN_MAX, TARGET_VARIABLE, color=COLOR_MIN_MAX
)

Zoomed view:

In [ ]:
plot_feature_target_relation(
    pdf[pdf["season"] == "20"], ALL_FEATURES_MIN_MAX, TARGET_VARIABLE, color=COLOR_MIN_MAX
)

### Feature distribution

In [ ]:
plot_feature_distribution(pdf, ALL_FEATURES_MIN_MAX, color=COLOR_MIN_MAX)

The following GIF compares the previously plotted feature distributions.

![GIF changes illustration](https://s8.gifyu.com/images/ezgif.com-gif-maker4847ef0cb3270edd.gif)

Before learning, some further useful visualizations are given.

**From now on, Min-Max transofrmed data will be used.**

## PCA (on min-max normalized data)

In [ ]:
from pyspark.ml.feature import PCA

PCA_NUM_COMPONENTS = 2

def perform_pca(df, num_components, input_col, output_col):
    pca = PCA(
        k=num_components, 
        inputCol=input_col, 
        outputCol=output_col
    )
    pca_model = pca.fit(df)

    return pca_model.transform(df), pca_model

In [ ]:
def plot_pca_explained_variance(pca_model, num_components_to_plot=2, figsize=(8,6)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    _ = sns.barplot(
        x=[i for i in range(num_components_to_plot)],
        y=pca_model.explainedVariance.values[0:num_components_to_plot],
        ax=ax,
        palette="summer"
    )

    _ = ax.set_xlabel("Eigenvalues", labelpad=16, fontsize=16)
    _ = ax.set_ylabel("Proportion of Variance", fontsize=16)
    _ = ax.set_xticklabels(
        [f"Principal Component {i}" for i in range(num_components_to_plot)], 
        rotation=0
    )
    _ = ax.set_title("Explained variance of each Principal Component")

Actual PCA:

In [ ]:
df, pca_model = perform_pca(
    df=df,
    num_components=PCA_NUM_COMPONENTS,
    input_col="all_vec_min_max",
    output_col="all_vec_min_max_pcs"
)

In [ ]:
plot_pca_explained_variance(
    pca_model=pca_model
)

#### Visualization

In [ ]:
pdf = df.toPandas()

In [ ]:
def scatter_plot(
    x,
    y,
    x_label,
    y_label,
    title="",
    c=None,
    c_map=plt.cm.get_cmap("tab10"),
    figsize=(12,8),
    ):
    
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    _ = plt.scatter(
        x=x,
        y=y,
        c=y if c is None else c,
        edgecolor="none",
        #alpha=1,
        cmap=c_map,
        axes=ax,
    )

    _ = ax.set_xlabel(x_label, labelpad=20, fontsize=16)
    _ = ax.set_ylabel(y_label, fontsize=16)
    _ = ax.set_title(title)

    plt.colorbar()
    plt.show()

In [ ]:
scatter_plot(
    x=pdf.all_vec_min_max_pcs.map(lambda x: x[0]),
    y=pdf.all_vec_min_max_pcs.map(lambda x: x[1]),
    c=pdf.points,
    x_label="Principal Component 0",
    y_label="Principal Component 1",
)  

PCA result is quite underwhelming: teams with widely different end-of-the-season placement tend to be mixed-up.

Notice that Principal Component Analysis is a linear model, so it is bounded to produce linear representations: usually, non-trivial problems present non-linear data.

For this reason, adding some non-linearity may possibly lead to a better lower-dimensional embedding...

In [ ]:
scatter_plot(
    x=pdf.all_vec_min_max_pcs.map(lambda x: x[0]),
    y=pdf.points,
    x_label="Principal Component 0",
    y_label="Points",
)
scatter_plot(
    x=pdf.all_vec_min_max_pcs.map(lambda x: x[1]),
    y=pdf.points,
    x_label="Principal Component 1",
    y_label="Points",
)

## t-SNE

t-SNE is one of the most used tools, when assuming non-linearity in the data space.

Implementation from SciKit Learn framework will be used, since PySpark does not provide any version of this tool.

SciKit Learn works with NumPy structures, so the feature vector in PySpark DataFrame must be converted to a NDArray structure.

In [ ]:
all_vec_min_max_np = np.array(
    list(
        map(
            lambda v: v["all_vec_min_max"].toArray(), 
            df.select("all_vec_min_max").collect()
        )
    )
)

points_np = np.array(
    list(
        map(
            lambda v: v["points"], 
            df.collect()
        )
    )
).reshape(-1)

Once data is ready, TNSE is called.

In [ ]:
from sklearn.manifold import TSNE

tsne_embedding = TSNE(
    n_components=2, learning_rate='auto', init='random', method="barnes_hut"
).fit_transform(all_vec_min_max_np)

And, once the embedding space is populated, the result is plotted

In [ ]:
scatter_plot(
    x=tsne_embedding[:,0],
    y=tsne_embedding[:,1],
    c=points_np,
    x_label="TSNE Embedding Dimension 0",
    y_label="TSNE Embedding Dimension 1",
)

In [ ]:
scatter_plot(
    x=tsne_embedding[:,0],
    y=points_np,
    x_label="TSNE Embedding Dimension 0",
    y_label="Points",
)
scatter_plot(
    x=tsne_embedding[:,1],
    y=points_np,
    x_label="TSNE Embedding Dimension 1",
    y_label="Points",
)

Unfortunately, adding non-linearity does not seem to improve the situation.

Not much progress has been made, w.r.t. PCA 

[TODO] comment on variance

## Comments on visualizations

Data seems to be:

* Not linearly separable.
* Suffering from high variance.
* ...

## Learning

The problem has been treated in two different forms:

* As **regression** on the **points**.
* As **classification** on the table/ranking areas (macro places).

In both cases, for all the different models will be used:

* **K-Fold Cross Validation**, to perform validation tests on the model performances.
* **Grid-search approach**, to perform hyperparameter tuning.

In [79]:
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.regression import Regressor
from pyspark.ml.regression import LinearRegressionTrainingSummary
from pyspark.ml.classification import Classifier
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import RandomForestClassificationTrainingSummary
from pyspark.ml.classification import MultilayerPerceptronClassificationSummary

from pyspark.ml.regression import RegressionModel
from pyspark.ml.classification import ClassificationModel
from pyspark.ml.classification import MultilayerPerceptronClassificationModel

In [76]:
def learn_best_model(
    estimator, 
    param_grid, 
    feature_vec, 
    label_col, 
    prediction_col,
    evaluation_metrics,
    evaluation_metrics_cv
):
    evaluators = dict()

    if isinstance(estimator, Regressor):
        evaluators = {
            metric: RegressionEvaluator(
                labelCol=label_col,
                predictionCol=prediction_col,
                metricName=metric,
            )
            for metric in evaluation_metrics
        }
    elif isinstance(estimator, Classifier) or isinstance(estimator, OneVsRest):
        evaluators = {
            metric: MulticlassClassificationEvaluator(
                labelCol=label_col,
                predictionCol=prediction_col,
                metricName=metric,
            )
            for metric in evaluation_metrics
        }
    else:
        raise Exception("Unexpected estimator, got" + str(type(estimator)))

    cross_validations = {
        metric: CrossValidator(
            estimator=estimator,
            estimatorParamMaps=param_grid,
            evaluator=evaluators[metric],
            numFolds=5,
            collectSubModels=True
        )
        for metric in evaluation_metrics_cv
    }

    cross_validated_models = dict()

    for metric in evaluation_metrics_cv:
        cross_validated_models[metric] = cross_validations[
            metric
        ].fit(train_df)

    return cross_validated_models, evaluators

In [77]:
def evaluate_learning_models(
    cross_validated_models, 
    evaluators,
    evaluation_metrics_cv
):
    for metric in evaluation_metrics_cv:
        best_model = cross_validated_models[metric].bestModel
        has_summary = True
        
        try:
            has_summary = best_model.hasSummary
        except AttributeError as e:
            # since the only models that have summary
            # have the hasSummary field,
            # it is needed to catch this exception
            has_summary = False
        
        if has_summary:
            training_result = best_model.summary
            
            if isinstance(training_result, LinearRegressionTrainingSummary):
                print(
                    f"""
                    Training set evaluation
                    {type(best_model).__name__}
                    Best model according to metric: {metric}
                    \tRMSE:        {builtins.round(training_result.rootMeanSquaredError, 2)}
                    \tR2:          {builtins.round(training_result.r2, 2)}
                    \tAdjusted R2: {builtins.round(training_result.r2adj, 2)}
                    """
                )
            elif isinstance(training_result, RandomForestClassificationTrainingSummary):
                print(
                    f"""
                    Training set evaluation
                    {type(best_model).__name__}
                    Best model according to metric: {metric}
                    \tAccuracy:                  {builtins.round(training_result.accuracy, 2)}
                    \tfalsePositiveRateByLabel:  {training_result.falsePositiveRateByLabel}
                    \tprecisionByLabel:          {training_result.precisionByLabel}
                    \trecallByLabel:             {training_result.recallByLabel}
                    \ttruePositiveRateByLabel:   {training_result.truePositiveRateByLabel}
                    \tweightedFalsePositiveRate: {training_result.weightedFalsePositiveRate}
                    \tweightedTruePositiveRate:  {training_result.weightedTruePositiveRate}
                    \tweightedPrecision:         {training_result.weightedPrecision}
                    \tweightedRecall:            {training_result.weightedRecall}
                    """
                )
            elif isinstance(best_model, MultilayerPerceptronClassificationModel):
                training_result = best_model.summary()

                print(
                    f"""
                    Training set evaluation
                    {type(best_model).__name__}
                    Best model according to metric: {metric}
                    \tAccuracy:                  {builtins.round(training_result.accuracy, 2)}
                    \tfalsePositiveRateByLabel:  {training_result.falsePositiveRateByLabel}
                    \tprecisionByLabel:          {training_result.precisionByLabel}
                    \trecallByLabel:             {training_result.recallByLabel}
                    \ttruePositiveRateByLabel:   {training_result.truePositiveRateByLabel}
                    \tweightedFalsePositiveRate: {training_result.weightedFalsePositiveRate}
                    \tweightedTruePositiveRate:  {training_result.weightedTruePositiveRate}
                    \tweightedPrecision:         {training_result.weightedPrecision}
                    \tweightedRecall:            {training_result.weightedRecall}
                    """
                )
            else:
                print(
                    f"""
                    Training set evaluation
                    {type(best_model).__name__}
                    Best model according to metric: {metric}
                    \tAccuracy: {builtins.round(training_result.accuracy, 2)}
                    \tF1:       {builtins.round(training_result.f1, 2)}
                    """
                )

            predictions = {
                metric: cross_validated_models[
                    metric
                ].bestModel.transform(test_df)
                for metric in evaluation_metrics_cv
            }

            for m, model in cross_validated_models.items():
                print(
                f"""
                Test set evaluation
                {type(best_model).__name__}
                Best model according to metric {m}
                """
                )

                for e, evaluator in evaluators.items():
                    print(f"{evaluator.getMetricName()}: {evaluator.evaluate(predictions[m])}")

        else: # no summary available
            for stage_name, stage_df in zip(["Train", "Test"], [train_df, test_df]):
                predictions = {
                    metric: best_model.transform(stage_df)
                    for metric in evaluation_metrics_cv
                }

                for m, model in cross_validated_models.items():
                    print(
                        f"""{stage_name} set evaluation
                        {type(model).__name__}
                        Best model elected by {m}"""
                    )

                    for e, evaluator in evaluators.items():
                        print(f"{evaluator.getMetricName()}: {evaluator.evaluate(predictions[m])}")


### Regression

In [ ]:
train_df, test_df = df.randomSplit([0.7, 0.3])

# TODO make upper case
evaluation_metrics = ["r2", "mse"]
evaluation_metrics_cv = ["r2"]

#### Linear Regression

In [ ]:
from pyspark.ml.regression import LinearRegression

FEATURES_COL = "all_vec_min_max"
LABEL_COL = "points"
PREDICTION_COL = "lr_predictions"

estimator = LinearRegression()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(lr_raw.regParam, [0.0, 0.001, 0.01, 0.1, 0.5, 1])
    # .addGrid(lr_raw.solver, ["auto", "normal", "l-bfgs"])
    # .addGrid(lr_raw.standardization, [True, False])
    # .addGrid(lr_raw.loss, ["squaredError", "huber"])
    # huber loss does NOT support ElasticNet Regularization :(
    # .addGrid(lr_raw.fitIntercept, [True, False])
    # .addGrid(lr_raw.elasticNetParam, [0.0, 0.25, 0.5, 1.0])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics = evaluation_metrics
)

In [ ]:
evaluate_learning_models(cross_validated_models, evaluators)

#### Decision Tree Regression

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

PREDICTION_COL = "dtr_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(estimator.maxDepth, [2, 5, 10])
    # .addGrid(estimator.maxBins, [16, 32, 64])
    # .addGrid(estimator.minInfoGain, [0, 0.1])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics = evaluation_metrics
)

In [ ]:
evaluate_learning_models(cross_validated_models, evaluators)

#### Gradient Boosted Tree Regression

In [ ]:
from pyspark.ml.regression import GBTRegressor

PREDICTION_COL = "gbtr_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(estimator.maxDepth, [2, 5, 10])
    # .addGrid(estimator.maxBins, [16, 32, 64])
    # .addGrid(estimator.minInfoGain, [0, 0.1])
    # .addGrid(estimator.subsamplingRate, [0.5, 1])
    # .addGrid(estimator.lossType, ["squared", "absolute"])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics = evaluation_metrics
)

In [ ]:
evaluate_learning_models(cross_validated_models, evaluators)

#### Random Forest Regression

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

PREDICTION_COL = "rfr_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(estimator.maxDepth, [2, 5, 10])
    # .addGrid(estimator.maxBins, [16, 32, 64])
    # .addGrid(estimator.minInfoGain, [0, 0.1])
    # .addGrid(estimator.subsamplingRate, [0.5, 1])
    # .addGrid(estimator.lossType, ["squared", "absolute"])
    # .addGrid(estimator.numTrees, [10, 20, 40])
    # .addGrid(estimator.featureSubsetStrategy, ["auto", "onethird", "all", "log2"])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics = evaluation_metrics
)

In [ ]:
evaluate_learning_models(cross_validated_models, evaluators)

### Classification

In [ ]:
FEATURES_COL = "all_vec_min_max"
LABEL_COLS = ["macro_place_naive", "macro_place_complex"]

#### Define macro places

The general idea of a macro place, is of a set of table partitioning.

In [104]:
NUM_MACRO_PLACES = 5

def get_macro_place(place, league, complex=False):
    def __naive(place, league):
        if league in ["Dutch Eredivisie", "German Bundesliga"]:
            if 1 <= place <= 3:
                return 0.0
            if 4 <= place <= 6:
                return 1.0
            if 7 <= place <= 10:
                return 2.0
            if 11 <= place <= 15:
                return 3.0
            if 16 <= place <= 18:
                return 4.0
        else:
            if 1 <= place <= 4:
                return 0.0
            if 5 <= place <= 8:
                return 1.0
            if 9 <= place <= 12:
                return 2.0
            if 13 <= place <= 16:
                return 3.0
            if 17 <= place <= 20:
                return 4.0

        return None

    def __complex(place, league):
        if league == "German Bundesliga":
            if 1 <= place <= 4:
                return 0.0
            if 5 <= place <= 7:
                return 1.0
            if 8 <= place <= 10:
                return 2.0
            if 11 <= place <= 15:
                return 3.0
            if 16 <= place <= 18:
                return 4.0

        if league == "Holland Eredivise":
            if place == 1:
                return 0.0
            if 2 <= place <= 3:
                return 1.0
            if 4 <= place <= 9:
                return 2.0
            if 10 <= place <= 15:
                return 3.0
            if 16 <= place <= 18:
                return 4.0

        if league == "French League 1":
            if 1 <= place <= 2:
                return 0.0
            if 3 <= place <= 5:
                return 1.0
            if 6 <= place <= 11:
                return 2.0
            if 12 <= place <= 17:
                return 3.0
            if 18 <= place <= 20:
                return  4.0
        
        else: #It, Sp, En
            if 1 <= place <= 4:
                return 0.0
            if 5 <= place <= 7:
                return 1.0
            if 8 <= place <= 12:
                return 2.0
            if 13 <= place <= 17:
                return 3.0
            if 18 <= place <= 20:
                return 4.0

        return None

    if complex:
        return __complex(place, league)

    return __naive(place, league)

get_macro_place_UDF = udf(
    lambda place, league, complex: get_macro_place(float(place), league, complex),
    DoubleType(),
)

In [105]:
df = df.withColumn(
    "macro_place_naive", get_macro_place_UDF(col("place"), col("league"), lit(False))
).withColumn(
    "macro_place_complex", get_macro_place_UDF(col("place"), col("league"), lit(True))
)

In [106]:
evaluation_metrics = ["accuracy", "f1"]
evaluation_metrics_cv = ["accuracy"]

train_df, test_df = df.randomSplit(
    [0.9,0.1]
    # [0.7, 0.3]
    # NOTE reactive 90/10 split, keep 70/30 just when using one league
)

#### Visualizations

In [ ]:
plot_feature_distribution(
    df.toPandas(), 
    ["macro_place_naive", "macro_place_complex"],
    color="teal",
    n_cols=2,
    figsize=(8,4)
)

In what follows comments:

* **Naive approach**: sligh imbalancement against the low partition of the table.
* **Compelx approach**: imbalancement in favor of the mid-table teams, confirming of course the visualizations in the beginning of the notebook.

#### SVM Classifier

In [ ]:
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LinearSVC

In [ ]:
PREDICTION_COL = "svmc_predictions"

estimator = OneVsRest(
    classifier=LinearSVC(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

In [ ]:
for label_col in LABEL_COLS:
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=LABEL_COL,
        prediction_col=PREDICTION_COL,
        evaluation_metrics = evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
for label_col in LABEL_COLS:
    evaluate_learning_models(
        cross_validated_models[label_col], 
        evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

#### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
FEATURES_COL = "all_vec_min_max"
LABEL_COLS = ["macro_place_naive", "macro_place_complex"]
PREDICTION_COL = "lrc_predictions"

estimator = OneVsRest(
    classifier=LogisticRegression(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, LABEL_COLS)
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

In [ ]:
cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
for label_col in LABEL_COLS:

    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col]
    )

#### Classification Tree

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

PREDICTION_COL = "dtc_predictions"

estimator = DecisionTreeClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

In [ ]:
for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics = evaluation_metrics
    )

In [ ]:
for label_col in LABEL_COLS:
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col]
    )

#### Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

PREDICTION_COL = "rfc_predictions"

estimator = RandomForestClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

In [ ]:
for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics = [
            "accuracy",
            "falsePositiveRateByLabel",
            "precisionByLabel",
            "recallByLabel",
            "truePositiveRateByLabel",
            "weightedFalsePositiveRate",
            "weightedTruePositiveRate",
            "weightedPrecision",
            "weightedRecall",
        ]
    )

In [ ]:
for label_col in LABEL_COLS:
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col]
    )

# Attempt 2: "less is more"

Ok, considering all features gives trash results.

What if we embrace the "less is more idea" and try to improve the results by means of using less features?

Nevertheless, feature correlation is very high, so, intrinsicly, it already did NOT make much sense to consider them all.

## Univariate Feature Selection

Log scaling reduces skewedness of "avg(mentality_penalties)" feature BUT it increases the skewedness of all the other features.
The other scalings (z-score and min-max) do NOT appear to be different than the "raw" data distribution.

For this reason and due to the limited amount of resources available on Google Colab, we decided to stick with the min-max scaled data.
In fact, the min-max scaling places "for free" all the features in the same scale, which is a very important consideration for SVM, which will be used in the upcoming sections

In [ ]:
from pyspark.ml.feature import UnivariateFeatureSelector

In [ ]:
selector = UnivariateFeatureSelector(
    featuresCol="all_vec_min_max",
    labelCol=TARGET_VARIABLE, 
    selectionMode="percentile"
).setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(0.08)

NUM_FEATURES = [2, 4, 6, 8]
THRESHOLDS = [num_features/24 for num_features in NUM_FEATURES]

In [ ]:
#df = df.drop("all_vec_min_max_ufs_2","all_vec_min_max_ufs_4","all_vec_min_max_ufs_6","all_vec_min_max_ufs_8")

In [ ]:
fit_result = dict()

for num_features, thr in zip(NUM_FEATURES, THRESHOLDS):
    selector.setSelectionThreshold(thr)
    selector.setOutputCol(f"all_vec_min_max_ufs_{num_features}"),
    fit_result[str(num_features)] = selector.fit(df)
    df = fit_result[str(num_features)].transform(df)

In [ ]:
selected_features = dict()
for num_features in NUM_FEATURES:
    selected_features[str(num_features)] = list(
        map(
            lambda i: ALL_FEATURES[i], fit_result[str(num_features)].selectedFeatures
        )
    )
    print(f"Univariate Feature Selection selected these {num_features} features:\n{selected_features[str(num_features)]}")

In [ ]:
pdf = df.toPandas()

In [ ]:
scatter_plot(
    x=pdf["all_vec_min_max_ufs_2"].map(lambda x: x[0]),
    y=pdf["all_vec_min_max_ufs_2"].map(lambda x: x[1]),
    c=pdf["points"],
    x_label="Feature 0",
    y_label="Feature 1",
)

In [ ]:
list(selected_features.keys())[-1]

In [ ]:
max_key = list(selected_features.keys())[-1]
plot_feature_distribution(
    pdf,
    selected_features[max_key], # last dictionary entry
    figsize=(10,4),
    color="orange",
    n_cols=(int(max_key) // 2)
)

### Feature-target relationship

In [ ]:
plot_feature_target_relation(
    pdf,
    selected_features[max_key],
    TARGET_VARIABLE,
    figsize=(10,8), 
    color=COLOR_MIN_MAX,
    n_cols=(int(max_key) // 4)
)

### Feature correlation

look at it from the original pearson matrix

state that these data distribs are trash.

So, for this reason, talk about overall and value.

## Overall as feature (min-max normalized)

Since the features are so correlated and performances of attempt 1 are trash, why not considering just the overall as a feature?

Maybe, we're lucky and the overall captures some other characteristics thay may steer the prediction a little bit more

In [ ]:
OVERALL = ["avg(overall)"]

COLOR_OVERALL_MIN_MAX = "green"

In [ ]:
assembler = VectorAssembler(
    inputCols=OVERALL, outputCol="overall_vec"
)

# min_max_df = assembler.transform(min_max_df)
df = assembler.transform(df)

In [ ]:
OVERALL_MIN_MAX = ["avg(overall)_min_max"]

In [ ]:
scaler = MinMaxScaler(
    inputCol="overall_vec", 
    outputCol="overall_vec_min_max"
)

In [ ]:
# min_max_df = scaler.fit(min_max_df).transform(min_max_df)
df = scaler.fit(df).transform(df)

In [ ]:
# pdf = min_max_df.toPandas()
pdf = df.toPandas()

pdf = feature_vec_to_cols(
    pdf=pdf,
    vec="overall_vec_min_max",
    columns=OVERALL_MIN_MAX
)

### Feature-target relationship

In [ ]:
plot_feature_target_relation(
    pdf,
    OVERALL_MIN_MAX,
    TARGET_VARIABLE,
    figsize=(10,4),
    color="chocolate",
    n_cols=1
)

### Feature distribution

In [ ]:
plot_feature_distribution(
    pdf,
    OVERALL_MIN_MAX,
    color="chocolate",
    figsize=(10,4),
    n_cols=1
)

## Value as feature (min-max normalized)

Oh, we found this paper: 

So, why not try their approach as well?

In [ ]:
VALUE = ["avg(value)"]

COLOR_VALUE_MIN_MAX = "coral"
VALUE_MIN_MAX = ["avg(value)_min_max"]

In [ ]:
assembler = VectorAssembler(
    inputCols=VALUE, outputCol="value_vec"
)

df = assembler.transform(df)

In [ ]:
scaler = MinMaxScaler(
    inputCol="value_vec", 
    outputCol="value_vec_min_max"
)

In [ ]:
df = scaler.fit(df).transform(df)

In [ ]:
pdf = df.toPandas()
pdf = feature_vec_to_cols(
    pdf=pdf,
    vec="value_vec_min_max",
    columns=VALUE_MIN_MAX
)

### Feature-target relationship

In [ ]:
plot_feature_target_relation(
    pdf,
    VALUE_MIN_MAX,
    TARGET_VARIABLE,
    figsize=(10,4),
    color=COLOR_VALUE_MIN_MAX,
    n_cols=1
)

Look, they are basically the same. In fact, if we check their correlation, we get... [high correlation on pearson]

### Feature distribution

In [ ]:
plot_feature_distribution(
    pdf,
    ["avg(value)"],
    color=COLOR_VALUE_MIN_MAX,
    figsize=(10,4),
    n_cols=1
)

Observe that there is a lot of skewness. We already know that the min_max normalization has the same exact curve.

Let's try to see what happens by applying a $log_2$ transformation.

In [ ]:
df = df.withColumn("value_log", to_log_UDF("avg(value)"))
pdf = df.toPandas()

In [ ]:
plot_feature_distribution(
    pdf,
    ["value_log"],
    color=COLOR_VALUE_MIN_MAX,
    figsize=(10,4),
    n_cols=1
)

In [ ]:
assembler = VectorAssembler(inputCols=["value_log"], outputCol="value_log_vec")
df = assembler.transform(df)

scaler = MinMaxScaler(
    inputCol="value_log_vec", 
    outputCol="value_log_vec_min_max"
)

df = scaler.fit(df).transform(df)
pdf = df.toPandas()

In [ ]:
pdf = feature_vec_to_cols(
    pdf=pdf,
    vec="value_log_vec_min_max",
    columns=["value_log_min_max"]
)

plot_feature_distribution(
    pdf,
    ["value_log_min_max"],
    color=COLOR_VALUE_MIN_MAX,
    figsize=(10,4),
    n_cols=1
)

Hint at a very high correlation, then show it with pearson matrix...

In [ ]:
pdf = feature_vec_to_cols(
    pdf=pdf,
    vec="overall_vec_min_max",
    columns=OVERALL_MIN_MAX
)

In [ ]:
pdf.columns

In [ ]:
plot_correlation_matrix(
    pdf, 
    ["avg(overall)_min_max", "value_log_min_max"],
    figsize=(8,4)
)

Comment this correlation, and move on with life

## Learning for attempt 2

*[
    state that via hyperparam grid we'll set the feature column, meaning that we'll try to train the models on all of the attemps

[state expected results, according to correlations and similar stuff]

]

In [ ]:
train_df, test_df = df.randomSplit([0.5, 0.5])

### Regression

In [ ]:
# TODO make upper case
evaluation_metrics = ["r2", "mse"]
evaluation_metrics_cv = ["r2"]

FEATURES_COL = "value_log_vec_min_max"
LABEL_COL = "points"

#### Linear Regression

In [ ]:
# lr_train_df, lr_test_df = feature_selection_train_df, feature_selection_test_df
lr_train_df, lr_test_df = train_df, test_df

In [ ]:
PREDICTION_COL = "lr_attempt2_predictions"

estimator = LinearRegression()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(lr_raw.regParam, [0.0, 0.001, 0.01, 0.1, 0.5, 1])
    # .addGrid(lr_raw.solver, ["auto", "normal", "l-bfgs"])
    # .addGrid(lr_raw.standardization, [True, False])
    # .addGrid(lr_raw.loss, ["squaredError", "huber"])
    # huber loss does NOT support ElasticNet Regularization :(
    # .addGrid(lr_raw.fitIntercept, [True, False])
    # .addGrid(lr_raw.elasticNetParam, [0.0, 0.25, 0.5, 1.0])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Decision Tree Regressor

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

PREDICTION_COL = "dtr_attempt2_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(estimator.maxDepth, [2, 5, 10])
    # .addGrid(estimator.maxBins, [16, 32, 64])
    # .addGrid(estimator.minInfoGain, [0, 0.1])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Random Forest Regressor

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

PREDICTION_COL = "rfr_attempt2_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(estimator.maxDepth, [2, 5, 10])
    # .addGrid(estimator.maxBins, [16, 32, 64])
    # .addGrid(estimator.minInfoGain, [0, 0.1])
    # .addGrid(estimator.subsamplingRate, [0.5, 1])
    # .addGrid(estimator.lossType, ["squared", "absolute"])
    # .addGrid(estimator.numTrees, [10, 20, 40])
    # .addGrid(estimator.featureSubsetStrategy, ["auto", "onethird", "all", "log2"])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Gradient Boosted Tree Regressor

In [ ]:
from pyspark.ml.regression import GBTRegressor

PREDICTION_COL = "gbtr_attempt2_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(estimator.maxDepth, [2, 5, 10])
    # .addGrid(estimator.maxBins, [16, 32, 64])
    # .addGrid(estimator.minInfoGain, [0, 0.1])
    # .addGrid(estimator.subsamplingRate, [0.5, 1])
    # .addGrid(estimator.lossType, ["squared", "absolute"])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

### Classification

In [ ]:
evaluation_metrics = ["accuracy", "f1"]
evaluation_metrics_cv = ["accuracy"]

#### SVM Classifier

In [ ]:
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LinearSVC

FEATURES_COL = "all_vec_min_max"
LABEL_COLS = ["macro_place_naive", "macro_place_complex"]
PREDICTION_COL = "svmc_predictions"

estimator = OneVsRest(
    classifier=LinearSVC(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=LABEL_COL,
        prediction_col=PREDICTION_COL,
        evaluation_metrics = evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
for label_col in LABEL_COLS:
    evaluate_learning_models(
        cross_validated_models[label_col], 
        evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

#### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

PREDICTION_COL = "lrc_attempt2_predictions"

estimator = OneVsRest(
    classifier=LogisticRegression(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, LABEL_COLS)
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
for label_col in LABEL_COLS:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

#### Classification Tree

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

PREDICTION_COL = "dtc_attempt2_predictions"

estimator = DecisionTreeClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
for label_col in LABEL_COLS:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

#### Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

PREDICTION_COL = "rfc_attempt2_predictions"

estimator = RandomForestClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics = [
            "accuracy",
            "falsePositiveRateByLabel",
            "precisionByLabel",
            "recallByLabel",
            "truePositiveRateByLabel",
            "weightedFalsePositiveRate",
            "weightedTruePositiveRate",
            "weightedPrecision",
            "weightedRecall",
        ],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
for label_col in LABEL_COLS:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

#### MLP

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

PREDICTION_COL = "mlpc_attempt2_predictions"

estimator = MultilayerPerceptronClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, ["value_log_vec_min_max"])
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .addGrid(estimator.layers, [[1, NUM_MACRO_PLACES]])
    .addGrid(estimator.solver, ["gd"])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
for label_col in LABEL_COLS:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

# Attempt 3: clustering

In [37]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import DenseVector, Vectors, VectorUDT
from pyspark.ml.evaluation import ClusteringEvaluator

In [38]:
def k_means(
    dataset,
    n_clusters,
    distance_measure="euclidean",
    max_iter=20,
    features_col="features",
    prediction_col="cluster",
):

    print(
        f"""Training K-means clustering using the following parameters: 
        - K (n. of clusters) = {n_clusters}
        - max_iter (max n. of iterations) = {max_iter}
        - distance measure = {distance_measure}
        """
    )

    # if distance_measure == "cosine":

        # NOTE we already start from normalized/std data, so, NO need to do it

        # Normalize inputs to unit-length vectors
        # dataset = Normalizer(
        #     inputCol=features_col, outputCol=features_col + "_norm", p=1
        # ).transform(dataset)

        # features_col = features_col + "_norm"

    # Train a K-means model
    kmeans = KMeans(
        featuresCol=features_col,
        predictionCol=prediction_col,
        k=n_clusters,
        initMode="k-means||",
        initSteps=5,
        tol=0.000001,
        maxIter=max_iter,
        distanceMeasure=distance_measure,
    )

    model = kmeans.fit(dataset)
    # here there are all the relevant clustering information

    # Make clusters
    clusters_df = model.transform(dataset)

    return model, clusters_df


def evaluate_k_means(
    clusters,
    metric_name="silhouette",
    distance_measure="squaredEuclidean",  # cosine
    prediction_col="cluster",
    featuresCol = "features"
):

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator(
        metricName=metric_name,
        distanceMeasure=distance_measure,
        predictionCol=prediction_col,
        featuresCol=featuresCol
    )

    return evaluator.evaluate(clusters)


def do_clustering(
    k_range, 
    input_df, 
    max_iter,
    featuresCol,
    clusterCol
):
    clustering_results = {}

    clusters_df = input_df

    # for k in tqdm( range(5, max_k_clusters + 1, 5), desc = "Performing clustering" ):
    # for k in tqdm(range(2, max_k_clusters + 1, 4), desc="Performing clustering"):
    for k in k_range:
        print(f"Running K-means using K = {k}")

        # model, clusters_df = k_means(tf_idf_df, k, max_iter=50, distance_measure="cosine") # Alternatively, distance_measure="euclidean"
        # model, clusters_df = k_means(input_df, k, max_iter=max_iter, distance_measure="cosine") # Alternatively, distance_measure="euclidean"
        model, clusters_df = k_means(
            # input_df,
            clusters_df, 
            k, 
            max_iter=max_iter, 
            distance_measure="cosine", 
            features_col=featuresCol,
            prediction_col=clusterCol + "_k_" + str(k),
        )  # Alternatively, distance_measure="euclidean"
        # silhouette_k = evaluate_k_means(clusters_df, distance_measure="cosine") # Alternatively, distance_measure="squaredEuclidean"
        silhouette_k = evaluate_k_means(
            clusters_df, 
            distance_measure="cosine",
            prediction_col=clusterCol + "_k_" + str(k),
            featuresCol=featuresCol

        )  # Alternatively, distance_measure="squaredEuclidean"
        wssd_k = model.summary.trainingCost
        # wssd_k = model.summary

        print(
            "Silhouette coefficient computed with cosine distance: {:.3f}".format(
                silhouette_k
            )
        )
        print(
            "Within-cluster Sum of Squared Distances (using cosine distance): {:.3f}".format(
                wssd_k
            )
        )
        print(
            "--------------------------------------------------------------------------------------"
        )

        l = list(
            enumerate(
                model.clusterCenters()
            )
        )
        l = [(ind, DenseVector(c)) for ind, c in l]
        # print(l)
        schema = ["cluster_id"  + "_k_" + str(k), "centroid"  + "_k_" + str(k)]

        schema = StructType([ 
            StructField("cluster_id" + "_k_" + str(k),IntegerType(),True), 
            StructField("centroid" + "_k_" + str(k),VectorUDT(),True), 
        ])
        centr_df = spark.createDataFrame(data=l, schema=schema)

        # centr_df.show()

        # df_with_centroids = clusters_df.join(
        #     centr_df, on=["cluster_id"]
        # )

        clusters_df = clusters_df.join(
            centr_df, on=["cluster_id" + "_k_" + str(k)]
        )



        clustering_results[str(k)] = {
            "silhouette_k"      : silhouette_k,
            "wssd_k"            : wssd_k,
            "model"             : model,
            "df"                : clusters_df,
            # "cluster_centroids" : model.clusterCenters(),
            # "centr_df" : centr_df,
            # "df_with_centroids" : df_with_centroids
        }

        
        

        # Free up memory space at the end of each iteration
        # del model
        # del clusters_df
        # gc.collect() # garbage collector
    
    clustering_results["df_with_centroid_full"] = clusters_df

    return clustering_results


def plot_clustering_results(clustering_results, k_range):

    # print(clustering_results)

    # k_col = list(clustering_results.keys())[:-1]
    k_col = [str(x) for x in k_range]
    wssd_col = [
        clustering_results[k]["wssd_k"] for k in k_col 
    ]
    silhouette_col = [
        clustering_results[k]["silhouette_k"] for k in k_col 
    ]

    # print(f"k_col: {k_col}")
    # print(wssd_col)
    # print(silhouette_col)

    plot_df_temp = pd.DataFrame([k_col, wssd_col, silhouette_col]).transpose()
    plot_df_temp.columns = ["K", "WSSD", "Silhouette"]
    # print(plot_df_temp)

    # Create a 1x1 figure
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    _ = sns.pointplot(
        data=plot_df_temp, x="K", y="WSSD", ax=ax, color="orangered"
    )
    _ = ax.set_xlabel("K")
    _ = ax.set_ylabel("WSSD")
    
    # Create a 1x1 figure
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    _ = sns.pointplot(
        data=plot_df_temp, x="K", y="Silhouette", ax=ax, color="orangered"
    )
    _ = ax.set_xlabel("K")
    _ = ax.set_ylabel("Silhouette")

In [39]:
import copy
from pyspark.ml.feature import VectorAssembler

CLUSTERING_FEATURES = copy.deepcopy(PLAYER_FEATURES)
CLUSTERING_FEATURES.remove("value")
CLUSTERING_FEATURES.remove("overall")

assembler = VectorAssembler(
    inputCols=CLUSTERING_FEATURES,
    outputCol="clustering_features"
)

pre_processed_df = assembler.transform(pre_processed_df)

In [40]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(
    inputCol="clustering_features",
    outputCol="clustering_features_min_max"    
)

pre_processed_df = scaler.fit(pre_processed_df).transform(pre_processed_df)

In [41]:
df_dict = {
    s: pre_processed_df.filter(col("season") == s) for s in seasons
}

MAX_K_CLUSTERS = 8
MAX_ITER = 20

k_range = [2]
# k_range = range(2, MAX_K_CLUSTERS, 4)
K_RANGE = [str(k) for k in k_range]

In [42]:
clustering_results_dict = dict()

for s in seasons:
    clustering_results_dict[s] = do_clustering(
        k_range=k_range, 
        input_df=df_dict[s], 
        max_iter=2,
        # max_iter=MAX_ITER,
        featuresCol="clustering_features_min_max",
        clusterCol="cluster_id",
    )


Running K-means using K = 2
Training K-means clustering using the following parameters: 
        - K (n. of clusters) = 2
        - max_iter (max n. of iterations) = 2
        - distance measure = cosine
        
Silhouette coefficient computed with cosine distance: 0.704
Within-cluster Sum of Squared Distances (using cosine distance): 14.332
--------------------------------------------------------------------------------------
Running K-means using K = 2
Training K-means clustering using the following parameters: 
        - K (n. of clusters) = 2
        - max_iter (max n. of iterations) = 2
        - distance measure = cosine
        
Silhouette coefficient computed with cosine distance: 0.858
Within-cluster Sum of Squared Distances (using cosine distance): 24.439
--------------------------------------------------------------------------------------


In [43]:
pre_processed_df = reduce(
    DataFrame.unionAll, 
    [
        clustering_results_dict[s]["df_with_centroid_full"] for s in seasons
    ]
)

## Clustering evaluation

### Evalutating single seasons

In [ ]:
for s in seasons:
    plot_clustering_results(clustering_results_dict[s], k_range=K_RANGE)

### Evaluating seasons all together

In [ ]:
clustering_results_dict["avg"] = dict.fromkeys(K_RANGE)

for k in clustering_results_dict["avg"].keys():
    clustering_results_dict["avg"][k] = {
        "wssd_k": 0,
        "silhouette_k": 0
    }

sum_wssd = dict.fromkeys(K_RANGE, 0)
sum_silhouette = dict.fromkeys(K_RANGE, 0)

for s in seasons:
    for k in K_RANGE:
        sum_wssd[k] += clustering_results_dict[s][k]["wssd_k"]
        sum_silhouette[k] += clustering_results_dict[s][k]["silhouette_k"]
    
avg_wssd = dict()
avg_silhouette = dict()

for k in K_RANGE:
    avg_wssd[k] = sum_wssd[k] / len(k_range)
    avg_silhouette[k] = sum_silhouette[k] / len(k_range)

    clustering_results_dict["avg"][k]["wssd_k"] = avg_wssd[k]
    clustering_results_dict["avg"][k]["silhouette_k"] = avg_silhouette[k]

In [ ]:
plot_clustering_results(
    clustering_results_dict["avg"], k_range=K_RANGE
)

## Dataset composition

### Compute distance between player and centroid of its cluster

In [44]:
compute_distance_from_centroid_UDF = udf(
    lambda player, centroid: float(
        Vectors.squared_distance(
            player, centroid
        )
    ), FloatType()
)

In [45]:
for k in K_RANGE:
    pre_processed_df = pre_processed_df.withColumn(
        "distance_from_centroid" + "_k_" + k,
        compute_distance_from_centroid_UDF(
            col("clustering_features_min_max"),
            col("centroid" + "_k_" + k)
        )
    )
        

### From players to teams

In [46]:
teams_df = pre_processed_df.groupBy(
    ["season", "club_name", "macro_role"]
).agg(
    {
        "distance_from_centroid" + "_k_" + str(k): "avg" for k in K_RANGE 
    }
)

for k in K_RANGE:
    teams_df = teams_df.withColumnRenamed(
        "avg(distance_from_centroid" + "_k_" + str(k) + ")",
        "avg_distance_from_centroid" + "_k_" + str(k)
    )

In [47]:
def generate_subquery(macro_role, k):
    return f"""(
        case
            when macro_role='{macro_role}' then avg_distance_from_centroid_k_{k} 
        else NULL
        end
    ) as avg_dist_macro_role_{int(macro_role)}_k_{k}
    """

In [48]:
teams_df.createOrReplaceTempView("t")

temp = dict()

for k in K_RANGE:
    temp[k] = (
        spark.sql(
            f"""
                select season, club_name, {generate_subquery(0.0, k)}, {generate_subquery(1.0, k)}, {generate_subquery(2.0, k)}, {generate_subquery(3.0, k)}, {generate_subquery(4.0, k)}, {generate_subquery(5.0, k)}, {generate_subquery(6.0, k)}, {generate_subquery(7.0, k)}
                from t
            """
        )
        .groupBy("season", "club_name")
        .agg(
            # TODO use for loop as in second cell of "from players to teams"
            sum(f"avg_dist_macro_role_0_k_{k}").alias(f"avg_dist_macro_role_0_k_{k}"),
            sum(f"avg_dist_macro_role_1_k_{k}").alias(f"avg_dist_macro_role_1_k_{k}"),
            sum(f"avg_dist_macro_role_2_k_{k}").alias(f"avg_dist_macro_role_2_k_{k}"),
            sum(f"avg_dist_macro_role_3_k_{k}").alias(f"avg_dist_macro_role_3_k_{k}"),
            sum(f"avg_dist_macro_role_4_k_{k}").alias(f"avg_dist_macro_role_4_k_{k}"),
            sum(f"avg_dist_macro_role_5_k_{k}").alias(f"avg_dist_macro_role_5_k_{k}"),
            sum(f"avg_dist_macro_role_6_k_{k}").alias(f"avg_dist_macro_role_6_k_{k}"),
            sum(f"avg_dist_macro_role_7_k_{k}").alias(f"avg_dist_macro_role_7_k_{k}"),
        )
    )

In [49]:
# NOTE executing this cell n times, w/o restoring teams_df --> 
# n copies of avg_dist_macro_role_[0:7]_k_[2, 6]

teams_df = temp[K_RANGE[0]]

for i in range(1, len(K_RANGE)):
    teams_df = teams_df.join(
        temp[K_RANGE[i]],
        on=["season", "club_name"]
    )

In [50]:
avg_distances_dict = dict()
avg_distances_vec_dict = dict()

for k in K_RANGE:
    
    avg_distances_dict[k] = [
        f"avg_dist_macro_role_{i}_k_{k}" for i in range(0, NUM_MACRO_ROLES)
    ]

    avg_distances_vec_dict[k] = f"avg_dist_vec_k_{k}"

In [51]:
import builtins
import itertools

global_max = teams_df.select(
    greatest(
        *list(
            itertools.chain.from_iterable(
                avg_distances_dict.values()
            )
        )
    ).alias("row_wise_max")
).collect()

global_max = [row["row_wise_max"] for row in global_max]

global_max = builtins.max(global_max)


In [52]:
teams_df = teams_df.fillna(global_max * 1.5)    

In [53]:
for k in K_RANGE:
    assembler = VectorAssembler(
        inputCols=avg_distances_dict[k], outputCol=avg_distances_vec_dict[k]
    )

    teams_df = assembler.transform(teams_df)

## Learning from clustering

In [54]:
df = df.join(
    teams_df,
    on=["club_name", "season"]
)

In [88]:
train_df, test_df = df.randomSplit(
    [0.9, 0.1]
    # NOTE reactive 90/10 split, keep 70/30 just when using one league
    # [0.7, 0.3]
)

In [56]:
NUM_FOLDS_CV = 4
FEATURES_COL = [f"avg_dist_vec_k_{k}" for k in K_RANGE]
LABEL_COL = "points"

### Regression

In [57]:
evaluation_metrics = ["r2", "mse"]
evaluation_metrics_cv = ["r2"]

#### Linear Regression

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "lr_attempt3_predictions"

estimator = LinearRegression()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(lr_raw.regParam, [0.0, 0.001, 0.01, 0.1, 0.5, 1])
    # .addGrid(lr_raw.solver, ["auto", "normal", "l-bfgs"])
    # .addGrid(lr_raw.standardization, [True, False])
    # .addGrid(lr_raw.loss, ["squaredError", "huber"])
    # huber loss does NOT support ElasticNet Regularization :(
    # .addGrid(lr_raw.fitIntercept, [True, False])
    # .addGrid(lr_raw.elasticNetParam, [0.0, 0.25, 0.5, 1.0])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Decision Tree Regressor

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "dtr_attempt3_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Random Forest

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "rfr_attempt3_predictions"

estimator = RandomForestRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Gradient Boosted Trees

In [ ]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "gbtr_attempt3_predictions"

estimator = GBTRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

### Classification

In [58]:
evaluation_metrics = ["accuracy", "f1"]
evaluation_metrics_cv = ["accuracy"]

#### SVM

In [70]:
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LinearSVC

FEATURES_COL = [f"avg_dist_vec_k_{k}" for k in K_RANGE]
LABEL_COLS = ["macro_place_naive", "macro_place_complex"]
PREDICTION_COL = "svmc_attempt3_predictions"

estimator = OneVsRest(
    classifier=LinearSVC(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

# for label_col in LABEL_COLS:
#     cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
#         estimator=estimator,
#         param_grid=param_grid,
#         feature_vec=[FEATURES_COL],
#         label_col=LABEL_COL,
#         prediction_col=PREDICTION_COL,
#         evaluation_metrics = evaluation_metrics,
#         evaluation_metrics_cv=evaluation_metrics_cv
#     )

In [ ]:
for label_col in LABEL_COLS:
    evaluate_learning_models(
        cross_validated_models[label_col], 
        evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

#### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

PREDICTION_COL = "lrc_attempt3_predictions"

estimator = OneVsRest(
    classifier=LogisticRegression(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, [FEATURES_COL])
    .addGrid(estimator.labelCol, LABEL_COLS)
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

# for label_col in LABEL_COLS:
#     print(f"Learning with label {label_col}")
#     cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
#         estimator=estimator,
#         param_grid=param_grid,
#         feature_vec=[FEATURES_COL],
#         label_col=label_col,
#         prediction_col=PREDICTION_COL,
#         evaluation_metrics=evaluation_metrics,
#         evaluation_metrics_cv=evaluation_metrics_cv
#     )

In [ ]:
for label_col in LABEL_COLS:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

#### Classification Tree

In [71]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "dtc_attempt3_predictions"

estimator = DecisionTreeClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

Learning with label macro_place_naive


Learning with label macro_place_complex


In [72]:
for label_col in LABEL_COLS:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

Evaluating with label macro_place_naive
Train set evaluation
                        CrossValidatorModel
                        Best model elected by accuracy


accuracy: 0.0
f1: 0.0
Test set evaluation
                        CrossValidatorModel
                        Best model elected by accuracy
accuracy: 0.0
f1: 0.0
Evaluating with label macro_place_complex
Train set evaluation
                        CrossValidatorModel
                        Best model elected by accuracy
accuracy: 0.0
f1: 0.0
Test set evaluation
                        CrossValidatorModel
                        Best model elected by accuracy


accuracy: 0.0
f1: 0.0


#### Random Forest

In [73]:
from pyspark.ml.classification import RandomForestClassifier

PREDICTION_COL = "rfc_attempt3_predictions"

estimator = RandomForestClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics = [
            "accuracy",
            "falsePositiveRateByLabel",
            "precisionByLabel",
            "recallByLabel",
            "truePositiveRateByLabel",
            "weightedFalsePositiveRate",
            "weightedTruePositiveRate",
            "weightedPrecision",
            "weightedRecall",
        ],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

Learning with label macro_place_naive


Learning with label macro_place_complex


In [74]:
for label_col in LABEL_COLS:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

Evaluating with label macro_place_naive

                    Training set evaluation
                    RandomForestClassificationModel
                    Best model according to metric: accuracy
                    	Accuracy:                  1.0
                    	falsePositiveRateByLabel:  [0.0, 0.0, 0.0]
                    	precisionByLabel:          [1.0, 1.0, 1.0]
                    	recallByLabel:             [1.0, 1.0, 1.0]
                    	truePositiveRateByLabel:   [1.0, 1.0, 1.0]
                    	weightedFalsePositiveRate: 0.0
                    	weightedTruePositiveRate:  1.0
                    	weightedPrecision:         1.0
                    	weightedRecall:            1.0
                    

                Test set evaluation
                RandomForestClassificationModel
                Best model according to metric accuracy
                
accuracy: 0.0
falsePositiveRateByLabel: 0.0
precisionByLabel: 0.0
recallByLabel: 0.0
truePositiveRateByLabe

weightedFalsePositiveRate: 0.0
weightedTruePositiveRate: 0.0
weightedPrecision: 0.0
weightedRecall: 0.0
Evaluating with label macro_place_complex

                    Training set evaluation
                    RandomForestClassificationModel
                    Best model according to metric: accuracy
                    	Accuracy:                  1.0
                    	falsePositiveRateByLabel:  [0.0, 0.0, 0.0]
                    	precisionByLabel:          [1.0, 1.0, 1.0]
                    	recallByLabel:             [1.0, 1.0, 1.0]
                    	truePositiveRateByLabel:   [1.0, 1.0, 1.0]
                    	weightedFalsePositiveRate: 0.0
                    	weightedTruePositiveRate:  1.0
                    	weightedPrecision:         1.0
                    	weightedRecall:            1.0
                    

                Test set evaluation
                RandomForestClassificationModel
                Best model according to metric accuracy
                
a

recallByLabel: 0.0
truePositiveRateByLabel: 0.0
weightedFalsePositiveRate: 0.0
weightedTruePositiveRate: 0.0
weightedPrecision: 0.0
weightedRecall: 0.0


#### MLP

In [94]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

PREDICTION_COL = "mlpc_attempt3_predictions"

estimator = MultilayerPerceptronClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, LABEL_COLS)
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .addGrid(estimator.layers, [[NUM_MACRO_ROLES, NUM_MACRO_PLACES]])
    .addGrid(estimator.solver, ["gd"])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in ["macro_place_naive"]:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=[
            "accuracy",
            "falsePositiveRateByLabel",
            "precisionByLabel",
            "recallByLabel",
            "truePositiveRateByLabel",
            "weightedFalsePositiveRate",
            "weightedTruePositiveRate",
            "weightedPrecision",
            "weightedRecall",
        ],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

Learning with label macro_place_naive


In [118]:
for label_col in ["macro_place_naive"]:
    print(f"Evaluating with label {label_col}")
    evaluate_learning_models(
        cross_validated_models=cross_validated_models[label_col],
        evaluators=evaluators[label_col],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

Evaluating with label macro_place_naive

                    Training set evaluation
                    MultilayerPerceptronClassificationModel
                    Best model according to metric: accuracy
                    	Accuracy:                  0.4
                    	falsePositiveRateByLabel:  [0.25, 0.0, 0.23076923076923078, 0.32, 0.0]
                    	precisionByLabel:          [0.2222222222222222, 0.0, 0.4, 0.5, 0.0]
                    	recallByLabel:             [0.2857142857142857, 0.0, 0.4444444444444444, 0.8, 0.0]
                    	truePositiveRateByLabel:   [0.2857142857142857, 0.0, 0.4444444444444444, 0.8, 0.0]
                    	weightedFalsePositiveRate: 0.20076923076923076
                    	weightedTruePositiveRate:  0.4
                    	weightedPrecision:         0.29015873015873017
                    	weightedRecall:            0.4
                    

                Test set evaluation
                MultilayerPerceptronClassificationModel

## Clustering-learning cross evaluation

Classic left-right plot, with:
Left Y --> elbow result
Right Y --> accuracy
X axis --> # clusters

# Attempt 4: thinking "Deep", shallow injecting some priors

In [108]:
rp_df = df

In [109]:
MAX_PLACE = 21

## Computing the prior (RP coefficient)

In [110]:
rp_df.select("season", "club_name", "place").createOrReplaceTempView("t")

In [111]:
rp_df = spark.sql(
    f"""
    select t.season, t.club_name,
        avg(
            (
                select sub.place
                where sub.season < t.season and sub.club_name == t.club_name
            )
        ) as rp_coeff
    from t, t as sub
    group by t.season, t.club_name
    order by t.season desc
    """
).fillna(MAX_PLACE)

In [112]:
rp_df = rp_df.withColumn("rp_coeff", MAX_PLACE - col("rp_coeff"))

In [113]:
ADD_RP_TRADEOFF = [0.5, 1, 1.5]

add_normalize_by_rp_UDF = udf(
    lambda points, rp, tradeoff: points + tradeoff * rp, DoubleType()
)

In [114]:
rp_df = rp_df.join(
    df, on=["club_name", "season"]
)

In [115]:
RP_NORMALIZED_COLS = [
    f"avg(overall)_rp_normalized_tradeoff_{tradeoff}".replace(
        ".", "-"
    ) for tradeoff in ADD_RP_TRADEOFF
]

for tradeoff, col_name in zip(ADD_RP_TRADEOFF, RP_NORMALIZED_COLS):
    rp_df = rp_df.withColumn(
        col_name, add_normalize_by_rp_UDF(
            col("avg(overall)"), col("rp_coeff"), lit(tradeoff)
        )
    )

    assembler = VectorAssembler(
        inputCols=[col_name], 
        outputCol=col_name + "_vec"
    )

    rp_df = assembler.transform(rp_df)

## Learning

In [116]:
train_df, test_df = rp_df.randomSplit(
    # [0.9, 0.1]
    [0.4, 0.6]
)

In [90]:
NUM_FOLDS_CV = 4
# TODO restore all values
FEATURES_COL = [RP_NORMALIZED_COLS[0] + "_vec"]
# FEATURES_COL = [
#     rp_normalized_col + "_vec" for rp_normalized_col in RP_NORMALIZED_COLS 
# ]
LABEL_COL = "points"

### Regression

In [74]:
evaluation_metrics = ["r2", "mse"]
evaluation_metrics_cv = ["r2"]

#### Linear Regression

In [82]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "lr_attempt4_predictions"

estimator = LinearRegression()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    # .addGrid(lr_raw.regParam, [0.0, 0.001, 0.01, 0.1, 0.5, 1])
    # .addGrid(lr_raw.solver, ["auto", "normal", "l-bfgs"])
    # .addGrid(lr_raw.standardization, [True, False])
    # .addGrid(lr_raw.loss, ["squaredError", "huber"])
    # huber loss does NOT support ElasticNet Regularization :(
    # .addGrid(lr_raw.fitIntercept, [True, False])
    # .addGrid(lr_raw.elasticNetParam, [0.0, 0.25, 0.5, 1.0])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [83]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)


                    Training set evaluation
                    LinearRegressionModel
                    Best model according to metric: r2
                    	RMSE:        11.3
                    	R2:          0.53
                    	Adjusted R2: 0.5
                    

                Test set evaluation
                LinearRegressionModel
                Best model according to metric r2
                
r2: 0.15630461159361153
mse: 390.26254586292924


#### Decision Tree Regressor

In [87]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "dtr_attempt4_predictions"

estimator = DecisionTreeRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [88]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

Train set evaluation
                        CrossValidatorModel
                        Best model elected by r2
r2: 0.9880442450048018
mse: 3.2450980392156867
Test set evaluation
                        CrossValidatorModel
                        Best model elected by r2
r2: 0.12146250404129544
mse: 406.37922705314014


#### Random Forest

In [93]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "rfr_attempt4_predictions"

estimator = RandomForestRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [94]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

Train set evaluation
                        CrossValidatorModel
                        Best model elected by r2


r2: 0.8144197176499781
mse: 65.9984081679894
Test set evaluation
                        CrossValidatorModel
                        Best model elected by r2
r2: 0.11078756153929059
mse: 364.4342346125731


#### Gradient Boosted Tree

In [95]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "gbtr_attempt4_predictions"

estimator = GBTRegressor()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models, evaluators = learn_best_model(
    estimator=estimator,
    param_grid=param_grid,
    feature_vec=[FEATURES_COL],
    label_col=LABEL_COL,
    prediction_col=PREDICTION_COL,
    evaluation_metrics=evaluation_metrics,
    evaluation_metrics_cv=evaluation_metrics_cv,
)

In [96]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

Train set evaluation
                        CrossValidatorModel
                        Best model elected by r2
r2: 0.9999852190669405


mse: 0.0052565824386878466
Test set evaluation
                        CrossValidatorModel
                        Best model elected by r2
r2: -0.1263864182428014
mse: 461.6374608084736


### Classification

In [97]:
evaluation_metrics = ["accuracy", "f1"]
evaluation_metrics_cv = ["accuracy"]

In [99]:
# TODO restore all values
# LABEL_COLS = ["macro_place_naive", "macro_place_complex"]
LABEL_COLS = ["macro_place_naive"]

#### SVM

In [100]:
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LinearSVC

PREDICTION_COL = "svmc_attempt4_predictions"

estimator = OneVsRest(
    classifier=LinearSVC(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

# for label_col in LABEL_COLS:
#     cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
#         estimator=estimator,
#         param_grid=param_grid,
#         feature_vec=[FEATURES_COL],
#         label_col=LABEL_COL,
#         prediction_col=PREDICTION_COL,
#         evaluation_metrics = evaluation_metrics,
#         evaluation_metrics_cv=evaluation_metrics_cv
#     )

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Logistic Regression

In [102]:
from pyspark.ml.classification import LogisticRegression

PREDICTION_COL = "lrc_attempt4_predictions"

estimator = OneVsRest(
    classifier=LogisticRegression(),
)

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, LABEL_COLS)
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

# for label_col in LABEL_COLS:
#     print(f"Learning with label {label_col}")
#     cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
#         estimator=estimator,
#         param_grid=param_grid,
#         feature_vec=[FEATURES_COL],
#         label_col=label_col,
#         prediction_col=PREDICTION_COL,
#         evaluation_metrics=evaluation_metrics,
#         evaluation_metrics_cv=evaluation_metrics_cv
#     )

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Classification Tree

In [118]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder

PREDICTION_COL = "dtc_attempt3_predictions"

estimator = DecisionTreeClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, LABEL_COLS)
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=evaluation_metrics,
        evaluation_metrics_cv=evaluation_metrics_cv
    )

Learning with label macro_place_naive


KeyboardInterrupt: 

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

PREDICTION_COL = "rfc_attempt3_predictions"

estimator = RandomForestClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, [LABEL_COL])
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in LABEL_COLS:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics = [
            "accuracy",
            "falsePositiveRateByLabel",
            "precisionByLabel",
            "recallByLabel",
            "truePositiveRateByLabel",
            "weightedFalsePositiveRate",
            "weightedTruePositiveRate",
            "weightedPrecision",
            "weightedRecall",
        ],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

#### MLP

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

PREDICTION_COL = "mlpc_attempt4_predictions"

estimator = MultilayerPerceptronClassifier()

param_grid = (
    ParamGridBuilder()
    .addGrid(estimator.featuresCol, FEATURES_COL)
    .addGrid(estimator.labelCol, LABEL_COLS)
    .addGrid(estimator.predictionCol, [PREDICTION_COL])
    .addGrid(estimator.layers, [[NUM_MACRO_ROLES, NUM_MACRO_PLACES]])
    .addGrid(estimator.solver, ["gd"])
    .build()
)

cross_validated_models = dict()
evaluators = dict()

for label_col in ["macro_place_naive"]:
    print(f"Learning with label {label_col}")
    cross_validated_models[label_col], evaluators[label_col] = learn_best_model(
        estimator=estimator,
        param_grid=param_grid,
        feature_vec=[FEATURES_COL],
        label_col=label_col,
        prediction_col=PREDICTION_COL,
        evaluation_metrics=[
            "accuracy",
            "falsePositiveRateByLabel",
            "precisionByLabel",
            "recallByLabel",
            "truePositiveRateByLabel",
            "weightedFalsePositiveRate",
            "weightedTruePositiveRate",
            "weightedPrecision",
            "weightedRecall",
        ],
        evaluation_metrics_cv=evaluation_metrics_cv
    )

In [ ]:
evaluate_learning_models(
    cross_validated_models=cross_validated_models,
    evaluators=evaluators,
    evaluation_metrics_cv=evaluation_metrics_cv
)

## RP impact

plot showing that the more the weight of the RP coefficient is increased, the more the accuracy ofc goes up